# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [7]:
# When using Google Colab we want to mount the data drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
except:
    IN_COLAB = False


Mounted at /content/drive


In [8]:
if IN_COLAB:
    !pip3 install -U scikit-learn

In [9]:
#Install kneed package if not there
kneed_installed = !pip list | grep kneed
if not kneed_installed:
    !pip install kneed

In [10]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
# from functools import reduce
from timeit import default_timer as timer
from datetime import datetime, timedelta
import gc

pd.options.mode.chained_assignment = None  # default='warn'

# magic word for producing visualizations in notebook
%matplotlib inline

# Google Colab environment:
if IN_COLAB:
    # Get pandas version
    print("Pandas version: ", pd.__version__)

Pandas version:  1.1.5


## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

## Strategy for exploring the data:

* Load datasets. Initially load random samples, finally load the full datasets.
* Load excel information files.
* Get the attributes from excel files and compare them to the attributes appearing in the datasets.
* Get the value pools for each known attribute in the excel files and check against the values for each attribute in the datasets.
* Replace the values outside the pools with NaNs.
* Replace values in the pools representing unknowns, missing, not given, etc. with NaNs.
* Explore NaNs, analysing and understanding missing values in the two datasets.
* Identify significance of categorical, ordinal attributes.

### Load data and excel files

In [11]:
# Start timer notebook run
start_run = timer()

# Current date/time suffix for filenames
ts = datetime.now().strftime("%Y%m%d_%H%M%S")

# figure bbox caption
cprt = '© 2021 · Chris Liatas'
# copyright bbox properties
bb_cprt_args = dict(boxstyle='square,pad=47.8', mutation_aspect=.015, facecolor='grey', alpha=0.3)

In [12]:
# Assign path names to variables for easier manipulation later on.
colab_dir = "/content/drive/MyDrive/workspace/arvato_data/"
remote_dir = "../../data/Term2/capstone/arvato_data/"

azdias_file = f"{colab_dir if IN_COLAB else remote_dir}Udacity_AZDIAS_052018.csv"
customers_file = f"{colab_dir if IN_COLAB else remote_dir}Udacity_CUSTOMERS_052018.csv"
mailout_train_file = f"{colab_dir if IN_COLAB else remote_dir}Udacity_MAILOUT_052018_TRAIN.csv"
mailout_test_file = f"{colab_dir if IN_COLAB else remote_dir}Udacity_MAILOUT_052018_TEST.csv"

# Files with information about the columns depicted in the csv files
dias_attrib = f"{colab_dir if IN_COLAB else remote_dir}DIAS Attributes - Values 2017.xlsx"
dias_info = f"{colab_dir if IN_COLAB else remote_dir}DIAS Information Levels - Attributes 2017.xlsx"

# Kaggle submission file
kg_submission = colab_dir if IN_COLAB else ""
# Kaggle competition
kg_competition = 'udacity-arvato-identify-customers'

We will create a function to load a sample of the datasets to speed things up during the data exploration phase, particularly for the larger azdias dataset. This will reduce the memory used for holding the whole dataset initially, but also greatly improve the speed of calculations when exploring and cleaning the data. We will later use the whole dataset for best results.


In [ ]:
def get_df_sample(n_samples, seed=None, *args, **kwargs):
    """Load a random sample out of an input csv file or the whole file.

    Args:
        n_samples (int): Size of sample (number of rows) to randomly load from the input file.
        args, kwargs: Pass arguments and keyword arguments as in pandas `read_csv` method.
        Ref: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

    Returns:
        The dataframe loaded from the input file of n_samples rows
    
    Reference:
        Stack Overflow - `Read a small random sample from a big CSV file into a Python data frame`:
        https://stackoverflow.com/a/61631765/10074873
        Pandas - `Iterating through files chunk by chunk`:
        https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#iterating-through-files-chunk-by-chunk
        Numpy - `Random Generator`:
        https://numpy.org/doc/stable/reference/random/generator.html#random-generator
    """
    rng = np.random.default_rng(seed=seed)

    samples_reader = pd.read_csv(*args, **kwargs)
    df = samples_reader.get_chunk(n_samples)

    for chunk in samples_reader:
        chunk.index = rng.integers(n_samples, size=len(chunk))
        df.loc[chunk.index] = chunk
    
    return df


* Read first the attributes as strings, to get the categories and then use that to read them directly as categorical.

In [ ]:
# Set to `True` to go through all the cells.
RUN_ALL = False

In [ ]:
if RUN_ALL:
    # Try loading a sample
    azdias_try1 = get_df_sample(200000, 42, azdias_file, sep=';', chunksize=256000)

    azdias_try1.head()

In [ ]:
if RUN_ALL:
    # Check the types of columns
    azdias_try1.dtypes.value_counts()

In [ ]:
if RUN_ALL:
    # Check `object` columns
    azdias_try1.select_dtypes(include=object).head()

####Check columns 18, 19 that produced warnings when reading csv data

In [ ]:
if RUN_ALL:
    # Check the two columns to identify why pandas "complained" about mixed types
    azdias_try1_1819 = azdias_try1.iloc[:,[18, 19]]

    azdias_try1_1819.columns

The two attributes are `'CAMEO_DEUG_2015', 'CAMEO_INTL_2015'`. We should check their values range to further explore them.

In [ ]:
if RUN_ALL:
    azdias_try1_1819.CAMEO_DEUG_2015.unique(), azdias_try1_1819.CAMEO_INTL_2015.unique()

In [ ]:
if RUN_ALL:
    # Clean up
    del azdias_try1, azdias_try1_1819

In these columns there are two values `X` and `XX` respectively that fall outside the range of values for this attributes as seen in `DIAS Attributes - Values 2017.xlsx`.

#### Importing values as NaNs at loading with `read_csv`

By observing the helper files `DIAS Attributes - Values 2017.xlsx` and `DIAS Information Levels - Attributes 2017.xlsx` we can see that zeros (`0` or `0.0`) and `-1` (`-1` or `-1.0`) values may be converted to NaNs as they offer no information even in the fields that are considered as valid input. `na_values` argument of `read_csv` method will be used for this to do this conversion during data import. This will leave the value `9` that is considered as `unknown` in some variables to be processed later.

We will take advantage of the `na_values` argument in `read_csv`method to pass a tuple of values that will:

* convert specific values to NaNs
* allow auto importing of attributes that were mixed types as specific types (numerical).

We will also use `parse_dates` argument to import attribute `EINGEFUEGT_AM` as datetime.

There are there more categorical attributes (`'CAMEO_DEU_2015', 'D19_LETZTER_KAUF_BRANCHE', 'OST_WEST_KZ'`) that we can identify which will be converted later on.

We will initially use samples of the supplied datasets to be able to explore the data and perform all necessary computations faster, preventing notebook crashes due to insufficient memory.

A sample of 200,000 rows will be used for `AZDIAS` and 150,000 rows for `CUSTOMERS` datasets to keep a relative balance between the two datasets.

In [ ]:
azd_sample_size = 300000
cus_sample_size = 150000

# Specific values to replace as np.NaN while loading the dataframe.
na_tupl = (-1, -1.0, 0, 0.0, 'X', 'XX')

# `read_csv` parameters dictionary
read_csv_kwargs = {
    'sep': ';',
    'na_values': na_tupl,
    'chunksize': 256000,
    'parse_dates': ['EINGEFUEGT_AM'],
}

# Read samples from azdias, customers datasets of size n rows, (use parse_dates to convert `EINGEFUEGT_AM` to datetime).
azdias = get_df_sample(azd_sample_size, None, azdias_file, **read_csv_kwargs)
# custrs = pd.read_csv(customers_file, sep=';', na_values=na_tupl, parse_dates=['EINGEFUEGT_AM'])
custrs = get_df_sample(cus_sample_size, None, customers_file, **read_csv_kwargs)

Let us check the dataframes now:

In [ ]:
azdias.shape, custrs.shape

((300000, 366), (150000, 369))

In [ ]:
azdias.sample(7)

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,CAMEO_DEU_2015,CAMEO_DEUG_2015,CAMEO_INTL_2015,CJT_GESAMTTYP,CJT_KATALOGNUTZER,CJT_TYP_1,CJT_TYP_2,CJT_TYP_3,CJT_TYP_4,CJT_TYP_5,CJT_TYP_6,D19_BANKEN_ANZ_12,D19_BANKEN_ANZ_24,D19_BANKEN_DATUM,D19_BANKEN_DIREKT,D19_BANKEN_GROSS,D19_BANKEN_LOKAL,D19_BANKEN_OFFLINE_DATUM,D19_BANKEN_ONLINE_DATUM,D19_BANKEN_ONLINE_QUOTE_12,D19_BANKEN_REST,D19_BEKLEIDUNG_GEH,D19_BEKLEIDUNG_REST,...,REGIOTYP,RELAT_AB,RETOURTYP_BK_S,RT_KEIN_ANREIZ,RT_SCHNAEPPCHEN,RT_UEBERGROESSE,SEMIO_DOM,SEMIO_ERL,SEMIO_FAM,SEMIO_KAEM,SEMIO_KRIT,SEMIO_KULT,SEMIO_LUST,SEMIO_MAT,SEMIO_PFLICHT,SEMIO_RAT,SEMIO_REL,SEMIO_SOZ,SEMIO_TRADV,SEMIO_VERT,SHOPPER_TYP,SOHO_KZ,STRUKTURTYP,TITEL_KZ,UMFELD_ALT,UMFELD_JUNG,UNGLEICHENN_FLAG,VERDICHTUNGSRAUM,VERS_TYP,VHA,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
9372,771330,NaN,7.0,NaN,NaN,NaN,NaN,NaN,20.0,4.0,NaN,NaN,1.0,1.0,NaN,4.0,3.0,6B,6.0,43.0,4.0,4.0,4.0,4.0,3.0,2.0,2.0,4.0,NaN,NaN,9,6.0,NaN,NaN,10,9,NaN,NaN,NaN,NaN,...,5.0,5.0,1.0,3.0,5.0,4.0,2,1,7,2,2,7,2,5,7,5,7,7,5,6,NaN,NaN,3.0,NaN,3.0,4.0,NaN,3.0,2.0,NaN,2.0,5.0,7.0,4.0,6.0,9.0,3.0,4,1,1
150835,482166,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.0,NaN,1.0,5.0,NaN,1.0,3.0,3D,3.0,25.0,3.0,5.0,1.0,2.0,5.0,5.0,5.0,5.0,NaN,NaN,10,NaN,NaN,NaN,10,10,NaN,NaN,NaN,NaN,...,NaN,1.0,5.0,3.0,5.0,2.0,3,4,4,4,3,4,7,4,4,3,4,6,4,7,3.0,NaN,3.0,NaN,1.0,5.0,NaN,4.0,1.0,NaN,NaN,8.0,11.0,11.0,6.0,9.0,2.0,1,1,4
92034,364325,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,1.0,10.0,NaN,4.0,1.0,8B,8.0,41.0,4.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,NaN,NaN,10,NaN,NaN,NaN,10,10,NaN,NaN,NaN,NaN,...,1.0,3.0,5.0,4.0,5.0,NaN,6,3,6,6,7,2,7,3,5,4,7,2,3,1,NaN,NaN,3.0,NaN,2.0,5.0,NaN,2.0,NaN,NaN,4.0,9.0,12.0,9.0,NaN,6.0,5.0,4,2,3
254110,525434,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,1.0,4.0,NaN,2.0,6.0,6B,6.0,43.0,4.0,3.0,3.0,3.0,4.0,2.0,2.0,3.0,NaN,NaN,10,NaN,NaN,NaN,10,10,NaN,NaN,NaN,NaN,...,3.0,3.0,2.0,4.0,4.0,3.0,7,6,3,5,6,3,6,3,3,4,1,1,3,2,2.0,1.0,2.0,NaN,1.0,3.0,NaN,NaN,1.0,NaN,3.0,8.0,11.0,10.0,4.0,9.0,2.0,3,2,3
179919,701438,NaN,3.0,20.0,NaN,NaN,NaN,NaN,20.0,1.0,NaN,NaN,6.0,1.0,NaN,4.0,6.0,2B,2.0,13.0,6.0,2.0,5.0,4.0,3.0,2.0,2.0,2.0,1.0,1.0,5,3.0,NaN,7.0,8,5,10.0,NaN,3.0,6.0,...,4.0,3.0,1.0,3.0,5.0,5.0,7,6,1,5,7,1,1,4,3,4,1,2,3,4,2.0,NaN,3.0,NaN,4.0,3.0,NaN,NaN,2.0,NaN,1.0,2.0,3.0,5.0,2.0,9.0,3.0,1,2,3
228622,600002,NaN,9.0,18.0,NaN,NaN,NaN,NaN,18.0,1.0,NaN,NaN,3.0,1.0,NaN,3.0,3.0,1D,1.0,15.0,1.0,1.0,5.0,5.0,1.0,3.0,1.0,2.0,NaN,NaN,10,NaN,NaN,NaN,10,10,NaN,NaN,NaN,NaN,...,1.0,2.0,1.0,3.0,5.0,5.0,7,5,2,7,7,2,2,3,5,6,4,5,7,4,3.0,NaN,3.0,NaN,4.0,5.0,1.0,5.0,2.0,NaN,3.0,7.0,7.0,4.0,2.0,7.0,3.0,1,2,2
277041,1054086,NaN,9.0,15.0,NaN,NaN,NaN,NaN,15.0,14.0,NaN,NaN,1.0,16.0,NaN,4.0,1.0,5A,5.0,31.0,5.0,2.0,4.0,3.0,2.0,2.0,3.0,3.0,NaN,NaN,10,NaN,NaN,NaN,10,10,NaN,NaN,NaN,NaN,...,7.0,3.0,5.0,1.0,2.0,4.0,6,6,1,7,7,1,6,2,5,4,3,5,3,4,2.0,NaN,3.0,NaN,4.0,4.0,NaN,5.0,2.0,NaN,4.0,8.0,11.0,8.0,6.0,9.0,3.0,6,2,3


In [ ]:
azdias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Columns: 366 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: datetime64[ns](1), float64(320), int64(42), object(3)
memory usage: 837.7+ MB


In [ ]:
custrs.sample(7)

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,CAMEO_DEU_2015,CAMEO_DEUG_2015,CAMEO_INTL_2015,CJT_GESAMTTYP,CJT_KATALOGNUTZER,CJT_TYP_1,CJT_TYP_2,CJT_TYP_3,CJT_TYP_4,CJT_TYP_5,CJT_TYP_6,D19_BANKEN_ANZ_12,D19_BANKEN_ANZ_24,D19_BANKEN_DATUM,D19_BANKEN_DIREKT,D19_BANKEN_GROSS,D19_BANKEN_LOKAL,D19_BANKEN_OFFLINE_DATUM,D19_BANKEN_ONLINE_DATUM,D19_BANKEN_ONLINE_QUOTE_12,D19_BANKEN_REST,D19_BEKLEIDUNG_GEH,D19_BEKLEIDUNG_REST,...,RT_KEIN_ANREIZ,RT_SCHNAEPPCHEN,RT_UEBERGROESSE,SEMIO_DOM,SEMIO_ERL,SEMIO_FAM,SEMIO_KAEM,SEMIO_KRIT,SEMIO_KULT,SEMIO_LUST,SEMIO_MAT,SEMIO_PFLICHT,SEMIO_RAT,SEMIO_REL,SEMIO_SOZ,SEMIO_TRADV,SEMIO_VERT,SHOPPER_TYP,SOHO_KZ,STRUKTURTYP,TITEL_KZ,UMFELD_ALT,UMFELD_JUNG,UNGLEICHENN_FLAG,VERDICHTUNGSRAUM,VERS_TYP,VHA,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
136940,728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN,10,NaN,NaN,NaN,10,10,NaN,NaN,NaN,NaN,...,4.0,5.0,NaN,6,3,6,6,7,3,5,5,5,4,7,2,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,COSMETIC_AND_FOOD,MULTI_BUYER,NaN,2,1
75636,157987,NaN,1.0,17.0,NaN,NaN,NaN,NaN,10.0,1.0,NaN,NaN,3.0,1.0,NaN,2.0,6.0,4A,4.0,22.0,5.0,5.0,1.0,1.0,5.0,5.0,5.0,5.0,1.0,1.0,5,3.0,NaN,NaN,5,10,NaN,NaN,NaN,7.0,...,1.0,5.0,3.0,3,3,5,2,3,4,7,1,4,1,2,6,4,7,3.0,NaN,1.0,NaN,3.0,5.0,NaN,NaN,1.0,1.0,4.0,1.0,1.0,1.0,2.0,9.0,7.0,1,COSMETIC,MULTI_BUYER,NaN,1,4
75953,3467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN,10,NaN,NaN,NaN,10,10,NaN,NaN,NaN,NaN,...,4.0,5.0,NaN,6,3,6,6,7,3,5,5,5,4,7,2,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,COSMETIC_AND_FOOD,MULTI_BUYER,NaN,1,2
54865,2272,2.0,2.0,16.0,NaN,NaN,NaN,NaN,11.0,1.0,NaN,NaN,2.0,1.0,NaN,2.0,6.0,2B,2.0,13.0,5.0,5.0,1.0,1.0,5.0,5.0,5.0,5.0,NaN,NaN,10,NaN,NaN,NaN,10,10,NaN,NaN,NaN,NaN,...,1.0,5.0,3.0,3,3,6,1,3,4,7,6,2,3,2,6,4,6,3.0,NaN,3.0,NaN,1.0,5.0,NaN,3.0,1.0,NaN,2.0,3.0,4.0,2.0,6.0,9.0,7.0,1,COSMETIC,MULTI_BUYER,NaN,1,4
105606,36931,1.0,1.0,8.0,NaN,NaN,NaN,NaN,8.0,1.0,NaN,NaN,1.0,1.0,NaN,3.0,6.0,1A,1.0,13.0,4.0,5.0,1.0,1.0,5.0,5.0,5.0,5.0,NaN,NaN,10,NaN,NaN,NaN,10,10,NaN,NaN,NaN,NaN,...,1.0,5.0,2.0,3,3,2,1,3,4,7,6,2,1,5,6,4,6,NaN,NaN,3.0,NaN,1.0,5.0,NaN,25.0,1.0,1.0,2.0,1.0,1.0,1.0,6.0,9.0,3.0,1,COSMETIC,SINGLE_BUYER,NaN,1,4
98948,109437,3.0,1.0,18.0,NaN,NaN,NaN,NaN,12.0,1.0,NaN,NaN,4.0,1.0,NaN,4.0,6.0,2C,2.0,14.0,4.0,4.0,2.0,2.0,4.0,4.0,5.0,4.0,NaN,NaN,9,NaN,6.0,NaN,10,10,NaN,7.0,NaN,2.0,...,1.0,5.0,2.0,4,4,7,4,4,7,4,7,4,4,6,7,5,7,1.0,NaN,3.0,NaN,4.0,2.0,1.0,9.0,1.0,NaN,NaN,3.0,5.0,4.0,2.0,9.0,3.0,2,FOOD,MULTI_BUYER,NaN,1,2
142772,11872,NaN,1.0,NaN,12.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0,1.0,NaN,1.0,5.0,4A,4.0,22.0,2.0,3.0,1.0,1.0,5.0,5.0,5.0,4.0,NaN,NaN,10,NaN,NaN,NaN,10,10,NaN,NaN,NaN,NaN,...,3.0,5.0,3.0,1,3,4,1,1,4,7,6,3,3,2,4,4,6,NaN,NaN,1.0,NaN,4.0,5.0,NaN,NaN,1.0,1.0,2.0,2.0,4.0,3.0,1.0,9.0,7.0,1,COSMETIC_AND_FOOD,MULTI_BUYER,NaN,1,4


In [ ]:
custrs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Columns: 369 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: datetime64[ns](1), float64(321), int64(42), object(5)
memory usage: 422.3+ MB


So, in `custrs` dataset the three extra columns (`'CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'`) are imported as one `float64` and two `object` dtypes, comparing to `azdias` columns.

In [ ]:
azd_obj_cols = azdias.select_dtypes(include=object)

azd_obj_cols.head()

,CAMEO_DEU_2015,D19_LETZTER_KAUF_BRANCHE,OST_WEST_KZ
0,6B,D19_VERSAND_REST,O
1,7B,D19_UNBEKANNT,W
2,5B,D19_UNBEKANNT,W
3,3C,D19_BUCH_CD,W
4,1B,D19_VERSAND_REST,O


In [ ]:
cus_obj_cols = custrs.select_dtypes(include=object)

cus_obj_cols.head()

,CAMEO_DEU_2015,D19_LETZTER_KAUF_BRANCHE,OST_WEST_KZ,PRODUCT_GROUP,CUSTOMER_GROUP
0,1A,D19_UNBEKANNT,W,COSMETIC_AND_FOOD,MULTI_BUYER
1,NaN,D19_BANKEN_GROSS,NaN,FOOD,SINGLE_BUYER
2,8A,D19_TELKO_REST,W,COSMETIC,MULTI_BUYER
3,NaN,NaN,NaN,COSMETIC_AND_FOOD,MULTI_BUYER
4,7B,D19_SCHUHE,W,FOOD,MULTI_BUYER


Convert the remaining **object** columns (`'CAMEO_DEU_2015', 'D19_LETZTER_KAUF_BRANCHE', 'OST_WEST_KZ', etc.`) to **categorical**:

In [ ]:
# Get the names of the columns to convert
azd_cat_cols = azd_obj_cols.columns.tolist()
cus_cat_cols = cus_obj_cols.columns.tolist()
# Convert all three columns using astype method
azdias[azd_cat_cols] = azdias[azd_cat_cols].astype('category')
custrs[cus_cat_cols] = custrs[cus_cat_cols].astype('category')

azdias[azd_cat_cols].dtypes

CAMEO_DEU_2015              category
D19_LETZTER_KAUF_BRANCHE    category
OST_WEST_KZ                 category
dtype: object

## Explore attributes in the dataset(s) and excel files

### 1. Compare attributes between the two excel files

Read in the excel files with attributes related information

In [ ]:
# Read in the mapping of data values for each feature in the DIAS dataset.
attr_cols = range(1,5)  # skip the first column as it is empty
dfdias_attrib = pd.read_excel(dias_attrib, header=1, usecols=attr_cols)
dfdias_attrib.head(7)

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
1,NaN,NaN,0,no classification possible
2,NaN,NaN,1,passive elderly
3,NaN,NaN,2,cultural elderly
4,NaN,NaN,3,experience-driven elderly
5,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
6,NaN,NaN,1,< 30 years


In [ ]:
# DIAS Information Levels - Attributes 2017.xlsx
dfdias_info = pd.read_excel(dias_info, header=1, usecols=attr_cols)
dfdias_info.head(7)

,Information level,Attribute,Description,Additional notes
0,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
1,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
2,NaN,ANREDE_KZ,gender,NaN
3,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
4,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...
5,NaN,FINANZ_SPARER,financial typology: money saver,NaN
6,NaN,FINANZ_VORSORGER,financial typology: be prepared,NaN


In [ ]:
# Get attributes' names from the Attribute column in the two excel files

# `DIAS Information Levels - Attributes 2017.xlsx`
dfdias_info_attrs = dfdias_info.Attribute.value_counts().index.values

# `DIAS Attributes - Values 2017.xlsx`
attrbf_names = dfdias_attrib.Attribute.value_counts().index.values

# Get the attribute names from `azdias` df columns:
azd_attr_names = azdias.columns.values

# Get the attribute names from `custrs` df columns:
cus_attr_names = custrs.columns.values

dfdias_info_attrs.size, attrbf_names.size, azd_attr_names.size, cus_attr_names.size

(313, 314, 366, 369)

In [ ]:
# Get the set intersection of the two arrays:
# We are comparing names as they were originally written in the files.
xcl_attr_intersc = np.intersect1d(attrbf_names, dfdias_info_attrs)

xcl_attr_intersc.size

300

In [ ]:
# Attributes values excel file contains some different attributes compared to information excel file:
xcl_attr_diff_vals_info = np.setdiff1d(attrbf_names, dfdias_info_attrs)

xcl_attr_diff_vals_info, xcl_attr_diff_vals_info.size

(array(['BIP_FLAG', 'D19_BANKEN_ANZ_12', 'D19_BANKEN_ANZ_24',
        'D19_GESAMT_ANZ_12', 'D19_GESAMT_ANZ_24', 'D19_LOTTO_RZ',
        'D19_TELKO_ANZ_12', 'D19_TELKO_ANZ_24', 'D19_VERSAND_ANZ_12',
        'D19_VERSAND_ANZ_24', 'D19_VERSI_ANZ_12', 'D19_VERSI_ANZ_24',
        'KBA13_CCM_3000', 'KBA13_CCM_3001'], dtype=object), 14)

In [ ]:
# Information excel file contains some different attributes compared to attributes excel file:
xcl_attr_diff_info_vals = np.setdiff1d(dfdias_info_attrs, attrbf_names)

xcl_attr_diff_info_vals, xcl_attr_diff_info_vals.size

(array(['ARBEIT', 'D19_BANKEN_ ANZ_12             D19_BANKEN_ ANZ_24',
        'D19_GESAMT_ANZ_12                                    D19_GESAMT_ANZ_24',
        'D19_TELKO_ ANZ_12                  D19_TELKO_ ANZ_24',
        'D19_VERSAND_ ANZ_12          D19_VERSAND_ ANZ_24',
        'D19_VERSI_ ANZ_12                                       D19_VERSI_ ANZ_24',
        'D19_VERSI_DATUM', 'D19_VERSI_OFFLINE_DATUM',
        'D19_VERSI_ONLINE_DATUM', 'EINWOHNER', 'GKZ', 'PLZ', 'PLZ8'],
       dtype=object), 13)

Taking the symmetric difference of the attributes in excel files, we can observe that there are more common attributes, but we must perform some string operations to bring them in similar forms for comparison.

In [ ]:
# Symmetric difference of the two attributes' sets.
xcl_attrs_symtrc_diff = np.union1d(xcl_attr_diff_vals_info, xcl_attr_diff_info_vals)

xcl_attrs_symtrc_diff, xcl_attrs_symtrc_diff.size

(array(['ARBEIT', 'BIP_FLAG',
        'D19_BANKEN_ ANZ_12             D19_BANKEN_ ANZ_24',
        'D19_BANKEN_ANZ_12', 'D19_BANKEN_ANZ_24', 'D19_GESAMT_ANZ_12',
        'D19_GESAMT_ANZ_12                                    D19_GESAMT_ANZ_24',
        'D19_GESAMT_ANZ_24', 'D19_LOTTO_RZ',
        'D19_TELKO_ ANZ_12                  D19_TELKO_ ANZ_24',
        'D19_TELKO_ANZ_12', 'D19_TELKO_ANZ_24',
        'D19_VERSAND_ ANZ_12          D19_VERSAND_ ANZ_24',
        'D19_VERSAND_ANZ_12', 'D19_VERSAND_ANZ_24',
        'D19_VERSI_ ANZ_12                                       D19_VERSI_ ANZ_24',
        'D19_VERSI_ANZ_12', 'D19_VERSI_ANZ_24', 'D19_VERSI_DATUM',
        'D19_VERSI_OFFLINE_DATUM', 'D19_VERSI_ONLINE_DATUM', 'EINWOHNER',
        'GKZ', 'KBA13_CCM_3000', 'KBA13_CCM_3001', 'PLZ', 'PLZ8'],
       dtype=object), 27)

We will fix the names of the attributes that seem different in the `DIAS Information Levels - Attributes 2017.xlsx` (or info) file.

* Parse lines containing more than one attribute

In [ ]:
# We will create a new list splitting the lines containing more than one attribute and stripping white space:
xcl_attr_diff_info_vals_fxd = [i.strip() for x in xcl_attr_diff_info_vals for i in x.split("  ", maxsplit=1)]

xcl_attr_diff_info_vals_fxd, len(xcl_attr_diff_info_vals_fxd)

(['ARBEIT',
  'D19_BANKEN_ ANZ_12',
  'D19_BANKEN_ ANZ_24',
  'D19_GESAMT_ANZ_12',
  'D19_GESAMT_ANZ_24',
  'D19_TELKO_ ANZ_12',
  'D19_TELKO_ ANZ_24',
  'D19_VERSAND_ ANZ_12',
  'D19_VERSAND_ ANZ_24',
  'D19_VERSI_ ANZ_12',
  'D19_VERSI_ ANZ_24',
  'D19_VERSI_DATUM',
  'D19_VERSI_OFFLINE_DATUM',
  'D19_VERSI_ONLINE_DATUM',
  'EINWOHNER',
  'GKZ',
  'PLZ',
  'PLZ8'],
 18)

* Expand the initial attributes list to include the newly identified attributes from above.

In [ ]:
# Our initial info attributes' list needs to be expanded to compensate for elements that were packed together.
dfdias_info_attrs_new = []
# Create a deep copy of the clean names list so it may be reused.
attrs_to_append = [i for i in xcl_attr_diff_info_vals_fxd]
# For every element in the imported attributes from information file check against the 
# fixed attribute names list and create a new list including fixed names.
for i in dfdias_info_attrs:
    if i in xcl_attr_diff_info_vals and len(attrs_to_append) > 0:
        for idx, x in enumerate(attrs_to_append):
            if i.startswith(x):
                if len(i) > len(x):
                    dfdias_info_attrs_new.append(''.join(x.split()))
                    dfdias_info_attrs_new.append(''.join(attrs_to_append[idx + 1].split()))
                    del attrs_to_append[idx + 1]
                    del attrs_to_append[idx]
                else:
                    dfdias_info_attrs_new.append(x)
                    del attrs_to_append[idx]
                break
    else:
        # just copy the rest of the elements.
        dfdias_info_attrs_new.append(i)

# Convert to numpy array
dfdias_info_attrs_new = np.array(dfdias_info_attrs_new)

dfdias_info_attrs_new[60:80], dfdias_info_attrs_new.size

(array(['D19_BEKLEIDUNG_GEH_RZ', 'PLZ8_ANTG3', 'KBA05_SEG3', 'KBA05_MOD1',
        'D19_REISEN_RZ', 'KBA13_HERST_FORD_OPEL', 'KBA05_CCM3',
        'KBA05_KRSKLEIN', 'FINANZTYP', 'CAMEO_DEU_2015',
        'D19_VERSICHERUNGEN_RZ', 'PLZ8_ANTG1', 'RETOURTYP_BK_S',
        'KBA13_CCM_0_1400', 'KBA05_ANTG2', 'KBA13_HALTER_66',
        'FINANZ_ANLEGER', 'KBA13_SEG_OBERKLASSE', 'EWDICHTE',
        'FINANZ_MINIMALIST'], dtype='<U27'), 318)

Now that we have fixed the names we can compare the attributes between the two attributes' information files.


In [ ]:
# Get the set intersection of the two arrays:
xcl_attr_intersc_fxd = np.intersect1d(attrbf_names, dfdias_info_attrs_new)

xcl_attr_intersc_fxd.size

310

Common attributes between the two files increased to 310. After this different attributes in each file become as follows:

In [ ]:
# DIAS Attributes - Values 2017.xlsx unique values:
xcl_attr_diff_vals_info = np.setdiff1d(attrbf_names, dfdias_info_attrs_new)

xcl_attr_diff_vals_info, xcl_attr_diff_vals_info.size

(array(['BIP_FLAG', 'D19_LOTTO_RZ', 'KBA13_CCM_3000', 'KBA13_CCM_3001'],
       dtype=object), 4)

In [ ]:
# DIAS Information Levels - Attributes 2017.xlsx contains some different attributes compared to attributes excel file:
xcl_attr_diff_info_vals = np.setdiff1d(dfdias_info_attrs_new, attrbf_names)

xcl_attr_diff_info_vals, xcl_attr_diff_info_vals.size

(array(['ARBEIT', 'D19_VERSI_DATUM', 'D19_VERSI_OFFLINE_DATUM',
        'D19_VERSI_ONLINE_DATUM', 'EINWOHNER', 'GKZ', 'PLZ', 'PLZ8'],
       dtype='<U27'), 8)

In [ ]:
# Symmetric difference of the two attributes' sets.
xcl_attrs_symtrc_diff = np.union1d(xcl_attr_diff_vals_info, xcl_attr_diff_info_vals)

xcl_attrs_symtrc_diff, xcl_attrs_symtrc_diff.size

(array(['ARBEIT', 'BIP_FLAG', 'D19_LOTTO_RZ', 'D19_VERSI_DATUM',
        'D19_VERSI_OFFLINE_DATUM', 'D19_VERSI_ONLINE_DATUM', 'EINWOHNER',
        'GKZ', 'KBA13_CCM_3000', 'KBA13_CCM_3001', 'PLZ', 'PLZ8'],
       dtype=object), 12)

In [ ]:
# Take the union for the total of unique values in both files together.
xcl_attrs_unique = np.union1d(attrbf_names, dfdias_info_attrs_new)

xcl_attrs_unique.size

322

The two Excel spreadsheets provided in the workspace (`DIAS Information Levels - Attributes 2017.xlsx` and `DIAS Attributes - Values 2017.xlsx`) contain 310 common attributes. In addition the first file contains 8 unique attributes while the second 4 unique attributes. In total there are 322 unique attributes present in both files together.

###Comparing `azdias`, `custrs` datasets to the unique attributes from the two excel files.


In [ ]:
attrs_intesc = np.intersect1d(azd_attr_names, xcl_attrs_unique)

azd_attr_names.size, xcl_attrs_unique.size, attrs_intesc.size

(366, 322, 276)

For the customers dataset, they only different attributes should be the 3 extra ones in the customers dataset, so the intersection should be the same as for `azdias` dataset:

In [ ]:
attrs_intesc = np.intersect1d(cus_attr_names, xcl_attrs_unique)

cus_attr_names.size, xcl_attrs_unique.size, attrs_intesc.size

(369, 322, 276)

At first look, we see that only 276 attributes are common. To explore what happens with the rest, we will look at the different attributes from the excel files.

In [ ]:
# Set difference of excel unique attributes - azdias set attributes will give us the different excel attributes:
difrnt_xcl_attrs = np.setdiff1d(xcl_attrs_unique, azd_attr_names)

difrnt_xcl_attrs, difrnt_xcl_attrs.size

(array(['BIP_FLAG', 'CAMEO_DEUINTL_2015', 'D19_BANKEN_DIREKT_RZ',
        'D19_BANKEN_GROSS_RZ', 'D19_BANKEN_LOKAL_RZ', 'D19_BANKEN_REST_RZ',
        'D19_BEKLEIDUNG_GEH_RZ', 'D19_BEKLEIDUNG_REST_RZ',
        'D19_BILDUNG_RZ', 'D19_BIO_OEKO_RZ', 'D19_BUCH_RZ',
        'D19_DIGIT_SERV_RZ', 'D19_DROGERIEARTIKEL_RZ', 'D19_ENERGIE_RZ',
        'D19_FREIZEIT_RZ', 'D19_GARTEN_RZ', 'D19_HANDWERK_RZ',
        'D19_HAUS_DEKO_RZ', 'D19_KINDERARTIKEL_RZ', 'D19_KK_KUNDENTYP',
        'D19_KOSMETIK_RZ', 'D19_LEBENSMITTEL_RZ', 'D19_LOTTO_RZ',
        'D19_NAHRUNGSERGAENZUNG_RZ', 'D19_RATGEBER_RZ', 'D19_REISEN_RZ',
        'D19_SAMMELARTIKEL_RZ', 'D19_SCHUHE_RZ', 'D19_SONSTIGE_RZ',
        'D19_TECHNIK_RZ', 'D19_TELKO_MOBILE_RZ', 'D19_TELKO_REST_RZ',
        'D19_TIERARTIKEL_RZ', 'D19_VERSAND_REST_RZ',
        'D19_VERSICHERUNGEN_RZ', 'D19_VOLLSORTIMENT_RZ',
        'D19_WEIN_FEINKOST_RZ', 'EINWOHNER', 'GEOSCORE_KLS7', 'GKZ',
        'HAUSHALTSSTRUKTUR', 'KBA13_CCM_1400_2500', 'PLZ', 'PLZ8',
        

Examining the `azdias` (and `customers`) columns' names we can observe that most of the above names are not actually missing but have been named slightly different. So, for example `CAMEO_INTL_2015` in azdias dataset is `CAMEO_DEUINTL_2015` in the attributes’ explanations file. This means we have to perform some operations to match the columns that appear different. We can create a dictionary to match old names with the new names, to rename the attributes' from excel files as in the azdias dataset. For most of the columns names with the prefix `D19_` if we remove the suffix `_RZ` we have the desired names.

We can also try to identify the rest of attributes, for example with `FLAG` in name and do the same for the rest, but not for `D19_` as we already know that only the suffix `_RZ` is missing from the name, hence it is seen as different string in the code comparisons.

In [ ]:
attr_flag = [x for x in azd_attr_names if 'FLAG' in x]
attr_bip = [x for x in azd_attr_names if 'BIP' in x]
attr_soho = [x for x in azd_attr_names if 'SOHO' in x]
attr_kba131400 = [x for x in azd_attr_names if 'KBA13_CCM_14' in x]
attr_geosc =[ x for x in azd_attr_names if 'GEO' in x]
attr_haus = [ x for x in azd_attr_names if 'HAUS' in x]
attr_wach = [ x for x in azd_attr_names if 'WACH' in x]
attr_gkz = [x for x in azd_attr_names if 'GKZ' in x]
attr_plz = [x for x in azd_attr_names if 'PLZ' in x]

attr_flag, attr_bip, attr_soho, attr_kba131400, attr_geosc, \
attr_haus, attr_wach, attr_gkz, attr_plz

(['DSL_FLAG', 'HH_DELTA_FLAG', 'UNGLEICHENN_FLAG'],
 [],
 ['SOHO_KZ'],
 ['KBA13_CCM_1400', 'KBA13_CCM_1401_2500'],
 ['KBA13_PEUGEOT'],
 ['ANZ_HAUSHALTE_AKTIV',
  'ANZ_STATISTISCHE_HAUSHALTE',
  'D19_HAUS_DEKO',
  'FINANZ_HAUSBAUER'],
 [],
 [],
 ['PLZ8_ANTG1',
  'PLZ8_ANTG2',
  'PLZ8_ANTG3',
  'PLZ8_ANTG4',
  'PLZ8_BAUMAX',
  'PLZ8_GBZ',
  'PLZ8_HHZ'])

From the above we see that we also have attributes `'SOHO_KZ', 'KBA13_CCM_1401_2500'` we can match. These matches will greatly reduce the number of different attributes in the azdias dataset when compared to the attributes' excel files. However, azdias dataset has some more attributes in addition to the ones present in the attributes files.

There are no additional attributes for `GKZ` and `PLZ` or `PLZ8` in azdias dataset.

In [ ]:
# Create a dictionary with key: value the old names replacing all `D19_*` names containing `_RZ`
xcl_attr_names_new = {i : i[:-3] if '_RZ' in i else i for i in difrnt_xcl_attrs}
# Replace SOHO_FLAG:
xcl_attr_names_new["SOHO_FLAG"] = 'SOHO_KZ'
# Replace CAMEO_DEUINTL_2015:
xcl_attr_names_new["CAMEO_DEUINTL_2015"] = 'CAMEO_INTL_2015'
# Replace KBA13_CCM_1400_2500:
xcl_attr_names_new["KBA13_CCM_1400_2500"] = 'KBA13_CCM_1401_2500'

xcl_attr_names_new.items()

dict_items([('BIP_FLAG', 'BIP_FLAG'), ('CAMEO_DEUINTL_2015', 'CAMEO_INTL_2015'), ('D19_BANKEN_DIREKT_RZ', 'D19_BANKEN_DIREKT'), ('D19_BANKEN_GROSS_RZ', 'D19_BANKEN_GROSS'), ('D19_BANKEN_LOKAL_RZ', 'D19_BANKEN_LOKAL'), ('D19_BANKEN_REST_RZ', 'D19_BANKEN_REST'), ('D19_BEKLEIDUNG_GEH_RZ', 'D19_BEKLEIDUNG_GEH'), ('D19_BEKLEIDUNG_REST_RZ', 'D19_BEKLEIDUNG_REST'), ('D19_BILDUNG_RZ', 'D19_BILDUNG'), ('D19_BIO_OEKO_RZ', 'D19_BIO_OEKO'), ('D19_BUCH_RZ', 'D19_BUCH'), ('D19_DIGIT_SERV_RZ', 'D19_DIGIT_SERV'), ('D19_DROGERIEARTIKEL_RZ', 'D19_DROGERIEARTIKEL'), ('D19_ENERGIE_RZ', 'D19_ENERGIE'), ('D19_FREIZEIT_RZ', 'D19_FREIZEIT'), ('D19_GARTEN_RZ', 'D19_GARTEN'), ('D19_HANDWERK_RZ', 'D19_HANDWERK'), ('D19_HAUS_DEKO_RZ', 'D19_HAUS_DEKO'), ('D19_KINDERARTIKEL_RZ', 'D19_KINDERARTIKEL'), ('D19_KK_KUNDENTYP', 'D19_KK_KUNDENTYP'), ('D19_KOSMETIK_RZ', 'D19_KOSMETIK'), ('D19_LEBENSMITTEL_RZ', 'D19_LEBENSMITTEL'), ('D19_LOTTO_RZ', 'D19_LOTTO'), ('D19_NAHRUNGSERGAENZUNG_RZ', 'D19_NAHRUNGSERGAENZUNG'), ('D19_

Let us rename the corresponding attribute names rows as per above dictionary and compare to `azdias` attributes again:

In [ ]:
# Create a copy of the `xcl_attrs_unique`
xcl_attrs_unique_new = pd.Series(xcl_attrs_unique)

# Rename specific attributes based on the created dictionary
xcl_attrs_unique_new.replace(xcl_attr_names_new, inplace=True)

# Compare to `azdias` attributes again taking the sets intersection
attrs_common = np.intersect1d(azd_attr_names, xcl_attrs_unique_new)

azd_attr_names.size, xcl_attrs_unique_new.size, attrs_common.size

(366, 322, 312)

There are 312 attributes in `azdias` dataset which are also present in the attributes explanatory excel files. Azdias dataset contains 54 additional attributes:

In [ ]:
# Let's see for the attributes' file what are the different attributes according to the previous intersection
azdias_only_attrs = np.setdiff1d(azd_attr_names, xcl_attrs_unique_new)

azdias_only_attrs, azdias_only_attrs.size

(array(['AKT_DAT_KL', 'ALTERSKATEGORIE_FEIN', 'ALTER_KIND1', 'ALTER_KIND2',
        'ALTER_KIND3', 'ALTER_KIND4', 'ANZ_KINDER',
        'ANZ_STATISTISCHE_HAUSHALTE', 'CJT_KATALOGNUTZER', 'CJT_TYP_1',
        'CJT_TYP_2', 'CJT_TYP_3', 'CJT_TYP_4', 'CJT_TYP_5', 'CJT_TYP_6',
        'D19_BUCH_CD', 'D19_KONSUMTYP_MAX', 'D19_LETZTER_KAUF_BRANCHE',
        'D19_SOZIALES', 'D19_TELKO_ONLINE_QUOTE_12',
        'D19_VERSI_ONLINE_QUOTE_12', 'DSL_FLAG', 'EINGEFUEGT_AM',
        'EINGEZOGENAM_HH_JAHR', 'EXTSEL992', 'FIRMENDICHTE', 'GEMEINDETYP',
        'HH_DELTA_FLAG', 'KBA13_ANTG1', 'KBA13_ANTG2', 'KBA13_ANTG3',
        'KBA13_ANTG4', 'KBA13_BAUMAX', 'KBA13_GBZ', 'KBA13_HHZ',
        'KBA13_KMH_210', 'KK_KUNDENTYP', 'KOMBIALTER', 'KONSUMZELLE',
        'LNR', 'MOBI_RASTER', 'RT_KEIN_ANREIZ', 'RT_SCHNAEPPCHEN',
        'RT_UEBERGROESSE', 'STRUKTURTYP', 'UMFELD_ALT', 'UMFELD_JUNG',
        'UNGLEICHENN_FLAG', 'VERDICHTUNGSRAUM', 'VHA', 'VHN', 'VK_DHT4A',
        'VK_DISTANZ', 'VK_ZG11'], dtype=obj

And `customers` dataframe has the above plus the three additional columns, as described in the project description.

In [ ]:
custrs_only_attrs = np.setdiff1d(cus_attr_names, xcl_attrs_unique_new)

custrs_only_attrs, custrs_only_attrs.size

(array(['AKT_DAT_KL', 'ALTERSKATEGORIE_FEIN', 'ALTER_KIND1', 'ALTER_KIND2',
        'ALTER_KIND3', 'ALTER_KIND4', 'ANZ_KINDER',
        'ANZ_STATISTISCHE_HAUSHALTE', 'CJT_KATALOGNUTZER', 'CJT_TYP_1',
        'CJT_TYP_2', 'CJT_TYP_3', 'CJT_TYP_4', 'CJT_TYP_5', 'CJT_TYP_6',
        'CUSTOMER_GROUP', 'D19_BUCH_CD', 'D19_KONSUMTYP_MAX',
        'D19_LETZTER_KAUF_BRANCHE', 'D19_SOZIALES',
        'D19_TELKO_ONLINE_QUOTE_12', 'D19_VERSI_ONLINE_QUOTE_12',
        'DSL_FLAG', 'EINGEFUEGT_AM', 'EINGEZOGENAM_HH_JAHR', 'EXTSEL992',
        'FIRMENDICHTE', 'GEMEINDETYP', 'HH_DELTA_FLAG', 'KBA13_ANTG1',
        'KBA13_ANTG2', 'KBA13_ANTG3', 'KBA13_ANTG4', 'KBA13_BAUMAX',
        'KBA13_GBZ', 'KBA13_HHZ', 'KBA13_KMH_210', 'KK_KUNDENTYP',
        'KOMBIALTER', 'KONSUMZELLE', 'LNR', 'MOBI_RASTER',
        'ONLINE_PURCHASE', 'PRODUCT_GROUP', 'RT_KEIN_ANREIZ',
        'RT_SCHNAEPPCHEN', 'RT_UEBERGROESSE', 'STRUKTURTYP', 'UMFELD_ALT',
        'UMFELD_JUNG', 'UNGLEICHENN_FLAG', 'VERDICHTUNGSRAUM', 'VHA',


In [ ]:
# Attributes unique to excel files.
xcl_only_attrs = np.setdiff1d(xcl_attrs_unique_new, azd_attr_names)

xcl_only_attrs, xcl_only_attrs.size

(array(['BIP_FLAG', 'D19_BUCH', 'D19_KK_KUNDENTYP', 'EINWOHNER',
        'GEOSCORE_KLS7', 'GKZ', 'HAUSHALTSSTRUKTUR', 'PLZ', 'PLZ8',
        'WACHSTUMSGEBIET_NB'], dtype=object), 10)

* `attrs_common` - 312 common attributes between `azdias` dataset and both excel files combined.
* `azdias_only_attrs` - 54 attributes appearing only in `azdias` dataset.
* `xcl_attrs_unique_new` - 322 unique attributes in both excel files combined.
* `xcl_only_attrs` - 10 attributes appearing only in excel files.

For 322 unique attributes present in both excel files combined there are 312 in common with the `azdias` dataset, leaving out 54 attributes known only in `azdias` dataset. Most of these seem to be related to attributes found in the excel files. For example `CJT` attribute in excel files seems to be related to the series of `CJT_TYP_*` attributes in azdias dataset. These are probably derived attributes to allow more granularity of collected information in certain fields.

In [ ]:
azdias["CJT_TYP_1"].value_counts()

5.0    89881
2.0    66243
3.0    57721
4.0    55071
1.0    29508
Name: CJT_TYP_1, dtype: int64

###Identify and replace out of range and unknown values with NaNs

We have already made some value replacements when loading the `.csv` files. We observe that some attributes include values outside the pool of values described in the information excel files. These values that are outside the allowed values described in the relevant excel files will be identified and replaced with NaNs. The process we will follow is:

* We will get the allowed values pool for each attribute from the excel file `DIAS Attributes - Values 2017.xlsx`.
* We will create a dictionary, mapping attribute names to value pools.
* We will in turn search the relevant columns in `azdias` and `customers` datasets and replace all the values outside these pools with NaNs.

In addition, there are some special cases where the allowed values are not specified, or the attributes do not have corresponding information in any of the excel files. These will have to be treated case by case.

In [ ]:
# In attributes values excel replace NaNs in attribute names with forward fill method ffill
# to make it easier to extract specific attribute names and values
dfdias_attrib_ffil = dfdias_attrib.fillna(method='ffill')

dfdias_attrib_ffil.head(7)

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
1,AGER_TYP,best-ager typology,0,no classification possible
2,AGER_TYP,best-ager typology,1,passive elderly
3,AGER_TYP,best-ager typology,2,cultural elderly
4,AGER_TYP,best-ager typology,3,experience-driven elderly
5,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
6,ALTERSKATEGORIE_GROB,age classification through prename analysis,1,< 30 years


1. Create a dictionary with attributes' names as keys and allowed input (`Value` column) as values.

In [ ]:
# First get the allowed value ranges per attribute from the attributes values file (DIAS Attributes - Values 2017.xlsx)
# Group by Attribute and get values in a multinex object, then transform to np.ndarray by chaining `values` method:
attrs_values_arr = dfdias_attrib_ffil.groupby("Attribute")["Value"].value_counts().index.values

# Create a helper function to parse values in the form '-1, 0'
def get_nums(x):
    """Return a list of integers from a string
    or the same input if not a string"""
    try:
        li = x.split(',')
        return list(map(int, li))
    except (AttributeError, ValueError):
        return [x]

# Create a dictionary mapping attributes as keys and allowed values in a list to key:[values]
# we are simultaneously injecting np.NaN in values' range as this might already be present
# among the attributes' values in `azdias`.
attrs_val_dict = {}
for i in attrs_values_arr:
    attrs_val_dict.setdefault(i[0],[np.NaN]).extend(get_nums(i[1]))

list(attrs_val_dict.items())[:2]

[('AGER_TYP', [nan, -1, 0, 1, 2, 3]),
 ('ALTERSKATEGORIE_GROB', [nan, 1, 2, 3, 4, 9, -1, 0])]

This dictionary must have its keys (attributes) names match the corresponding attributes names in `azdias` dataset for the cases where the attributes have differences in names, according to the previously created dictionary mapping `xcl_attr_names_new`:

In [ ]:
# We will use the `xcl_attr_names_new` to get the corresponding names and create a new dictionary.
attrs_val_pools = {xcl_attr_names_new.get(k, k): v for k, v in attrs_val_dict.items()}

We will create a different dictionary with the special cases with key values equal to `'…'`. These are some special cases where the input is described as "numeric value" which is attribute context related. So, this can be a year, a number typically coded from 1-10, or 1-3, but with the range not exactly specified. This will allow us to process these cases separately.

In [ ]:
# Create new dict while removing the related items from the previous
# Iterate through a list of keys 
attrs_val_dict_any = {k: attrs_val_pools.pop(k) for k in list(attrs_val_pools) if '…' in attrs_val_pools[k]}

attrs_val_dict_any

{'ANZ_HAUSHALTE_AKTIV': [nan, '…'],
 'ANZ_HH_TITEL': [nan, '…'],
 'ANZ_PERSONEN': [nan, '…'],
 'ANZ_TITEL': [nan, '…'],
 'GEBURTSJAHR': [nan, '…'],
 'KBA13_ANZAHL_PKW': [nan, '…'],
 'MIN_GEBAEUDEJAHR': [nan, '…']}

We now have two dictionaries mapping attributes to possible values for each attribute. There are some unique attributes (`azdias_only_attrs`) in `azdias` dataset that have not been accounted for. For these we will adopt the mapping of similar attributes based on context and add them in the first mapping dictionary. We can identify at least two value categories `CJT_*` and `KBA13_` that can be assigned to values' range list `[-1, 0, 1, 2, 3, 4, 5]` as can be observed from similar attributes in excel attributes values file. We cannot do the same for `D19_*` attributes or any of the rest attributes in the `azdias_only_attrs`.

In [ ]:
# we are then left with an undefined values range list of attributes for unique to `azdias` as:
azdias_only_attrs_undef = [i for i in azdias_only_attrs if not i.startswith(('CJT_', 'KBA13_'))]

azdias_only_attrs_undef = np.array(azdias_only_attrs_undef)

azdias_only_attrs_undef, azdias_only_attrs_undef.size

(array(['AKT_DAT_KL', 'ALTERSKATEGORIE_FEIN', 'ALTER_KIND1', 'ALTER_KIND2',
        'ALTER_KIND3', 'ALTER_KIND4', 'ANZ_KINDER',
        'ANZ_STATISTISCHE_HAUSHALTE', 'D19_BUCH_CD', 'D19_KONSUMTYP_MAX',
        'D19_LETZTER_KAUF_BRANCHE', 'D19_SOZIALES',
        'D19_TELKO_ONLINE_QUOTE_12', 'D19_VERSI_ONLINE_QUOTE_12',
        'DSL_FLAG', 'EINGEFUEGT_AM', 'EINGEZOGENAM_HH_JAHR', 'EXTSEL992',
        'FIRMENDICHTE', 'GEMEINDETYP', 'HH_DELTA_FLAG', 'KK_KUNDENTYP',
        'KOMBIALTER', 'KONSUMZELLE', 'LNR', 'MOBI_RASTER',
        'RT_KEIN_ANREIZ', 'RT_SCHNAEPPCHEN', 'RT_UEBERGROESSE',
        'STRUKTURTYP', 'UMFELD_ALT', 'UMFELD_JUNG', 'UNGLEICHENN_FLAG',
        'VERDICHTUNGSRAUM', 'VHA', 'VHN', 'VK_DHT4A', 'VK_DISTANZ',
        'VK_ZG11'], dtype='<U26'), 39)

So, for the `customers` the respective array will be:

In [ ]:
cus_only_attrs_undef = np.append(azdias_only_attrs_undef,
                                 ['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'])

cus_only_attrs_undef, cus_only_attrs_undef.size

(array(['AKT_DAT_KL', 'ALTERSKATEGORIE_FEIN', 'ALTER_KIND1', 'ALTER_KIND2',
        'ALTER_KIND3', 'ALTER_KIND4', 'ANZ_KINDER',
        'ANZ_STATISTISCHE_HAUSHALTE', 'D19_BUCH_CD', 'D19_KONSUMTYP_MAX',
        'D19_LETZTER_KAUF_BRANCHE', 'D19_SOZIALES',
        'D19_TELKO_ONLINE_QUOTE_12', 'D19_VERSI_ONLINE_QUOTE_12',
        'DSL_FLAG', 'EINGEFUEGT_AM', 'EINGEZOGENAM_HH_JAHR', 'EXTSEL992',
        'FIRMENDICHTE', 'GEMEINDETYP', 'HH_DELTA_FLAG', 'KK_KUNDENTYP',
        'KOMBIALTER', 'KONSUMZELLE', 'LNR', 'MOBI_RASTER',
        'RT_KEIN_ANREIZ', 'RT_SCHNAEPPCHEN', 'RT_UEBERGROESSE',
        'STRUKTURTYP', 'UMFELD_ALT', 'UMFELD_JUNG', 'UNGLEICHENN_FLAG',
        'VERDICHTUNGSRAUM', 'VHA', 'VHN', 'VK_DHT4A', 'VK_DISTANZ',
        'VK_ZG11', 'CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'],
       dtype='<U26'), 42)

* We now have `attrs_val_pools` which is a dictionary mapping of known attribute names and 
lists with accepted values per variable.
* `attrs_val_dict_any` which is a dictionary with attributes' names that do not have specific values defined but instead accept a wider range of numerical values.
* `azdias_only_attrs_undef`, `cus_only_attrs_undef` which are two arrays with all attributes known in `azdias` and `customers` only that do not have associated information about their accepted values.


####Replace unknown and specific values with NaNs

Using the `attrs_val_pools` we can check against the values in respective attributes in `azdias` and `customers` datasets. If values are found outside the accepted ones, these will be replaced by NaNs. We will use the keys in this dictionary to iterate through the attributes and make changes only in the common attributes (`attrs_common`) between `azdias` dataset and excel files.

In [ ]:
# for every key (attribute) in the dictionary created earlier
for k in attrs_val_pools:
    # if the attribute is among the common ones
    if k in attrs_common:
        # select all those rows in the column that fall outside the attribute's
        # specified values pool and replace them with NaN.
        azdias.loc[~azdias[k].isin(attrs_val_pools[k]), k] = np.NaN
        custrs.loc[~custrs[k].isin(attrs_val_pools[k]), k] = np.NaN

azdias['CAMEO_DEUG_2015'].value_counts()

8.0    45507
9.0    36443
6.0    35479
4.0    35077
3.0    29259
2.0    28097
7.0    25994
5.0    18443
1.0    11960
Name: CAMEO_DEUG_2015, dtype: int64

We want to find the attributes where 9 is used for unknown/NaN values, to replace it in `azdias`, `custrs` datasets with NaNs.

In [ ]:
# We will use the forward filled attribute dataframe to make it easier to extract the attributes' names
attrs_nine_name = dfdias_attrib_ffil[dfdias_attrib_ffil["Value"] == '-1, 9']["Attribute"].tolist()

# Create a dict to use as input for the pandas replace method
replace9_nan_dict = {k:9 for k in attrs_nine_name}
# Add 'KOMBIALTER' attribute in the dict that is not in excel files
replace9_nan_dict['KOMBIALTER'] = 9

# We can now use this dictionary of attributes to replace value 9 (unknown) with
# NaN for specific attributes in the azdias, custrs dataframe
azdias.replace(replace9_nan_dict, np.NaN, inplace=True)
custrs.replace(replace9_nan_dict, np.NaN, inplace=True)

Special cases where values' pool was `'…'`:

* `ANZ_HAUSHALTE_AKTIV` - number of households in the building numeric value, typically coded from 1-10 but higher values can be accepted (large building blocks)
* `ANZ_HH_TITEL` - number of academic title holder in building, numeric value typically coded from 1-10 but higher values can be accepted (large building blocks)
* `ANZ_PERSONEN` - number of adult persons in the household, numeric value typically coded from 1-3 but higher values can be accepted depending on the area and family size. This has some extreme values values in the range [23, 45] that may be proven outliers.
* `ANZ_TITEL` - number of professional title holders in a household, numeric value typically coded from 1-10 but higher values can be accepted depending on the family size.
* `GEBURTSJAHR` - year of birth, with values ranging from 1900 to 2017.
* `KBA13_ANZAHL_PKW` - number of cars in the PLZ8, numeric value. This may also have a large range based on the area (postcode) rural, urban areas.
* `MIN_GEBAEUDEJAHR` - year the building was first mentioned in the database, numeric values of years ranging from 1985 to 2016.

By observation these attributes seem to have acceptable values related to their context.

For the attributes known only to `azdias` dataset that cannot be related to existing information from the excel files there is a number of attributes that appear to be related to one another - `'ALTERSKATEGORIE_FEIN', 'ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4'` - and by context might provide categorization for number of children for age group bins. From `ALTERSKATEGORIE_FEIN` we can understand that there are 25 bins that correspond to an age range. `ANZ_KINDER` provides the number of children, while `''ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4'`, must provide the ages for the 1st, 2nd, 3rd and 4th child. `ANZ_STATISTISCHE_HAUSHALTE` seems to provide some budget categorization in a range 1 - 449 with a mean 7.6.

`'D19_BUCH_CD'` is probably related to books and or CDs ownership, `'D19_KONSUMTYP_MAX'` is a categorization of consumer type,`'D19_LETZTER_KAUF_BRANCHE'` provides a categorization based on recent products purchases, `'D19_SOZIALES'` probably provides a social type related categorization, `'D19_TELKO_ONLINE_QUOTE_12'` (few records) is telecoms related probably for the last 12 months average spending, `'D19_VERSI_ONLINE_QUOTE_12'` (few records) is another 12 months spending record, `'DSL_FLAG'` is probably internet line capacity related (DSL line) indicating if person has DSL (value 1), `'EINGEFUEGT_AM'` is probably the date and time the database entry for this entry was inserted (all times are set to 00:00:00), `'EINGEZOGENAM HH JAHR'` is possibly the year someone moved to a household, `'EXTSEL992'` is some kind of age related attribute, `'FIRMENDICHTE'` is a company size categorization probably related to the company each person works for, `'GEMEINDETYP'` is a community type categorization, `'HH_DELTA_FLAG'` cannot be associated with something known, `'KK_KUNDENTYP'` is a customer type categorization, `'KOMBIALTER'` is another age related categorization, `'KONSUMZELLE'` is another flag probably indicating somekind of consumer feature, `'MOBI_RASTER'` should be an attribute related to mobiles (or mobility) range 1-6, `'RT_KEIN_ANREIZ'` refers lack of incentives on something with values range 1-5, `'RT_SCHNAEPPCHEN'` describes bargain response on something with values range 1-5, `'RT_UEBERGROESSE'` describes over-size response on something with values range 1-5, `'SOHO_KZ'` could be related to small office / home office binary attribute, `'STRUKTURTYP'` provides structure type for homes (?) with values range 1-3, `'UMFELD_ALT'` records old relatives(?) with values range 1-5, `'UMFELD_JUNG'` records young relatives with values range 1-5, `'UNGLEICHENN_FLAG'` binary attribute recording unequality, `'VERDICHTUNGSRAUM'` probably describes the density of the living area (neibourhood, block?) values 1-45, `'VHA', 'VHN', 'VK_DHT4A', 'VK_DISTANZ','VK_ZG11'` cannot be identified.

In [ ]:
# Based on the above these are additional categoricals:
extra_cat_cols = np.array(['ALTERSKATEGORIE_FEIN', 'D19_BUCH_CD', 'D19_KONSUMTYP_MAX',
                          'D19_SOZIALES', 'D19_TELKO_ONLINE_QUOTE_12', 'D19_VERSI_ONLINE_QUOTE_12',
                          'FIRMENDICHTE', 'GEMEINDETYP', 'KK_KUNDENTYP', 'KOMBIALTER',
                          'MOBI_RASTER', 'RT_KEIN_ANREIZ', 'RT_SCHNAEPPCHEN',
                          'RT_UEBERGROESSE', 'STRUKTURTYP', 'UMFELD_ALT', 'UMFELD_JUNG',
                          'KONSUMZELLE', 'SOHO_KZ', 'UNGLEICHENN_FLAG', 'VHA', 'VHN',
                          'VK_DHT4A', 'VK_DISTANZ', 'VK_ZG11'])
# ('KOMBIALTER' needs to remove 9 as NaN)

All known categorical (includes all ordinal) attributes `cat_attrs`:

In [ ]:
cat_attrs = np.union1d(attrs_common, extra_cat_cols)

cat_attrs.size

336

##Analysing and understanding missing values in `azdias` and `custrs` dfs

* Total NaNs:

In [ ]:
# count NaNs in the dataframes
azdias_nan = azdias.isna().sum().sum()
cus_df_nan = custrs.isna().sum().sum()

(azdias.size, azdias_nan, round(azdias_nan/azdias.size, 2)), \
(custrs.size, cus_df_nan, round(cus_df_nan/custrs.size, 2))

((109800000, 30782105, 0.28), (55350000, 19690956, 0.36))

There are about 28% NaNs in `azdias` df and about 35-36% NaNs in the `customers` 
dataset out of the total data sizes.

* NaNs per column:

In [ ]:
def cols_nans_info(df, cols=None):
    """Return a new dataframe with NaNs summary per column of the input df

    Creates and returns a new dataframe with three columns that correspond to
    the attributes (columns) of the initial dataframe, the NaN counts per column
    and portion of NaNs per column.

    Args:
        df (pd.DataFrame): The input dataframe to process.
        cols (tuple): 3 elements tuple with the names of the three columns in the
        output dataframe. If left None the three default names ('Columns',
        'NaNcounts', 'NaNportions') will be used.
    
    Returns:
        A new dataframe with three columns providing counts and portions of NaNs
        per column in the input df. Example:

            Columns	     NaNcounts	NaNportions
        0	AGER_TYP	   685843	    0.769554
        1	AKT_DAT_KL     73499	    0.082470
        2	ALTER_HH	   310267	    0.348137
        3	ALTER_KIND1	   810163	    0.909048
        4	ALTER_KIND2	   861722	    0.966900
    """
    cols = cols or ('Columns', 'NaNcounts', 'NaNportions')
    new_df = df.isna().sum().rename_axis(cols[0]).reset_index(name=cols[1])
    new_df[cols[2]] = new_df[cols[1]] / df.shape[0]

    print('Average NaNs per column: ', round(new_df[cols[2]].mean(), 4))
    print('Max NaNs per column: ', round(new_df[cols[2]].max(), 4))
    print('Min NaNs per column: ', round(new_df[cols[2]].min(), 4))

    return new_df

# Get NaNs info for the `azdias` dataframe
print("AZDIAS")
azdias_nans = cols_nans_info(azdias)
# Get NaNs info for the `custrs` dataframe
print("CUSTOMERS")
cus_df_nans = cols_nans_info(custrs)

pd.merge(azdias_nans, cus_df_nans, on='Columns', suffixes=("_azd", "_cus"))

AZDIAS
Average NaNs per column:  0.2803
Max NaNs per column:  0.9991
Min NaNs per column:  0.0
CUSTOMERS
Average NaNs per column:  0.3558
Max NaNs per column:  0.999
Min NaNs per column:  0.0


,Columns,NaNcounts_azd,NaNportions_azd,NaNcounts_cus,NaNportions_cus
0,LNR,0,0.000000,0,0.000000
1,AGER_TYP,230705,0.769017,75850,0.505667
2,AKT_DAT_KL,24857,0.082857,36528,0.243520
3,ALTER_HH,104758,0.349193,53876,0.359173
4,ALTER_KIND1,272842,0.909473,140828,0.938853
...,...,...,...,...,...
361,WOHNDAUER_2008,24857,0.082857,36528,0.243520
362,WOHNLAGE,33900,0.113000,39977,0.266513
363,ZABEOTYP,0,0.000000,0,0.000000
364,ANREDE_KZ,0,0.000000,0,0.000000


A first look at the output indicates that there are attributes with a lot of NaNs and even some full of NaNs. Let us visualize these results to get a better picture of the data. We will create a histogram dividing the percentage of NaNs in 10 bins counting corresponding attributes in each bin.

In [ ]:
# Save image location
nan_col_distrb_file = '/content/drive/MyDrive/workspace/arvato_data/nan_col_distrb_{}.png'.format(ts) \
if IN_COLAB else 'nan_col_distrb_{}.png'.format(ts)

with sns.axes_style("whitegrid"):
    fig, ax = plt.subplots(1, 2, figsize=(10,5))
    sns.histplot(ax=ax[0], data=azdias_nans, x="NaNportions", bins=np.linspace(0, 1, 11)).set(title='[AZDIAS] Column counts per NaN portions')
    sns.histplot(ax=ax[1], data=cus_df_nans, x="NaNportions", bins=np.linspace(0, 1, 11)).set(title='[CUSTOMERS] Column counts per NaN portions')
    # copyright text
    fig.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
    fig.tight_layout()
    fig.savefig(nan_col_distrb_file, transparent=False, bbox_inches="tight")
    plt.close(fig)

We can see that both datasets have similar distributions of NaNs. The first plot for `azdias` shows that there are about 50+ attributes with NaNs above 80%. Around 280 attributes have less than 30% NaNs while the rest are between 30% - 80% NaNs. The second for `customers` shows that there are around 235 attributes with less than 30% NaNs and around 50 with more than 80% NaNs.

* `AZDIAS`


In [ ]:
azd_col_nan_bins = azdias_nans.NaNportions.value_counts(bins=np.linspace(0, 1, 11), sort=False)

azd_col_nan_bins = azd_col_nan_bins.rename_axis("Bins").reset_index(name="Count")

azd_col_nan_bins["ColPortion"] = azd_col_nan_bins.Count / azdias.shape[1]

azd_col_nan_bins

,Bins,Count,ColPortion
0,"(-0.001, 0.1]",67,0.183060
1,"(0.1, 0.2]",169,0.461749
2,"(0.2, 0.3]",40,0.109290
3,"(0.3, 0.4]",4,0.010929
4,"(0.4, 0.5]",10,0.027322
5,"(0.5, 0.6]",7,0.019126
6,"(0.6, 0.7]",6,0.016393
7,"(0.7, 0.8]",10,0.027322
8,"(0.8, 0.9]",21,0.057377
9,"(0.9, 1.0]",32,0.087432


In [ ]:
azd_col_nan_bins.loc[:2,].sum()

Count         276.000000
ColPortion      0.754098
dtype: float64

* `CUSTOMERS`

In [ ]:
cus_col_nan_bins = cus_df_nans.NaNportions.value_counts(bins=np.linspace(0, 1, 11), sort=False)

cus_col_nan_bins = cus_col_nan_bins.rename_axis("Bins").reset_index(name="Count")

cus_col_nan_bins["ColPortion"] = cus_col_nan_bins.Count / custrs.shape[1]

cus_col_nan_bins

,Bins,Count,ColPortion
0,"(-0.001, 0.1]",59,0.159892
1,"(0.1, 0.2]",0,0.000000
2,"(0.2, 0.3]",176,0.476965
3,"(0.3, 0.4]",40,0.108401
4,"(0.4, 0.5]",20,0.054201
5,"(0.5, 0.6]",10,0.027100
6,"(0.6, 0.7]",8,0.021680
7,"(0.7, 0.8]",9,0.024390
8,"(0.8, 0.9]",18,0.048780
9,"(0.9, 1.0]",29,0.078591


In [ ]:
cus_col_nan_bins.loc[:2,].sum()

Count         235.000000
ColPortion      0.636856
dtype: float64

####Format data for plotting

We need to convert the dataframe to [long-form](http://seaborn.pydata.org/tutorial/data_structure.html#long-form-vs-wide-form-data) to use the `hue` argument in `sns.barplot` to plot the data.

In [ ]:
merge_col_nan_bins = pd.merge(azd_col_nan_bins, cus_col_nan_bins, on='Bins', suffixes=("_azd", "_cus"))

merge_col_nan_bins.head()

,Bins,Count_azd,ColPortion_azd,Count_cus,ColPortion_cus
0,"(-0.001, 0.1]",67,0.183060,59,0.159892
1,"(0.1, 0.2]",169,0.461749,0,0.000000
2,"(0.2, 0.3]",40,0.109290,176,0.476965
3,"(0.3, 0.4]",4,0.010929,40,0.108401
4,"(0.4, 0.5]",10,0.027322,20,0.054201


In [ ]:
cols_nan_portns_melt = merge_col_nan_bins[
                                          ['Bins', 'ColPortion_azd', 'ColPortion_cus']
                                         ].melt(id_vars=('Bins'))
cols_nan_portns_melt.head()

,Bins,variable,value
0,"(-0.001, 0.1]",ColPortion_azd,0.183060
1,"(0.1, 0.2]",ColPortion_azd,0.461749
2,"(0.2, 0.3]",ColPortion_azd,0.109290
3,"(0.3, 0.4]",ColPortion_azd,0.010929
4,"(0.4, 0.5]",ColPortion_azd,0.027322


In [ ]:
# Save image location
nan_col_perc_file = '/content/drive/MyDrive/workspace/arvato_data/nan_col_perc_{}.png'.format(ts) \
if IN_COLAB else 'nan_col_perc_{}.png'.format(ts)

with sns.axes_style("whitegrid"):
    plt.figure(figsize=(8,5))
    sns.barplot(data=cols_nan_portns_melt, x="value", y="Bins", hue="variable", palette="Blues_d").set(title='[AZDIAS, CUSTOMERS] Columns portions per NaN bin')
    # copyright text
    plt.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
    plt.tight_layout()
    plt.savefig(nan_col_perc_file, transparent=False, bbox_inches="tight")
    plt.close()

###Rows NaNs analysis

* Identify missing values per row as percentage (portions).
* Drop rows with all NaNs (zero rows will be dropped):

In [ ]:
def rows_nans_info(df, cols=None):
    """Return a new dataframe with NaNs summary per row of the input df

    Creates and returns a new dataframe with three columns that correspond to
    the rows of the input dataframe, the NaN counts per row
    and portion of NaNs per row.

    Args:
        df (pd.DataFrame): The input dataframe to process.
        cols (tuple): 3 elements tuple with the names of the three columns in the
        output dataframe. If left None the three default names ('Rows',
        'NaNcounts', 'NaNportions') will be used.
    
    Returns:
        A new dataframe with three columns providing counts and portions of NaNs
        per row in the input df. Example:

            Rows    NaNcounts	NaNportions
        0	0	    685843	    0.769554
        1	1       73499	    0.082470
        2	2	    310267	    0.348137
        3	3	    810163	    0.909048
        4	4       861722	    0.966900
    """
    cols = cols or ('Rows', 'NaNcounts', 'NaNportions')
    new_df = df.isna().sum(axis=1).rename_axis(cols[0]).reset_index(name=cols[1])
    new_df[cols[2]] = new_df[cols[1]] / df.shape[1]

    print("Average NaNs per row: ", round(new_df[cols[2]].mean(), 4))
    print("Max NaNs per row: ", round(new_df[cols[2]].max(), 4))
    print("Min NaNs per row: ", round(new_df[cols[2]].min(), 4))

    return new_df

* `AZDIAS`

In [ ]:
azdias_row_nans = rows_nans_info(azdias)

azdias_row_nans.NaNportions.describe()

Average NaNs per row:  0.2803
Max NaNs per row:  0.8579
Min NaNs per row:  0.071


count    300000.000000
mean          0.280347
std           0.198368
min           0.071038
25%           0.183060
50%           0.213115
75%           0.243169
max           0.857923
Name: NaNportions, dtype: float64

* `CUSTOMERS`

In [ ]:
custrs_row_nans = rows_nans_info(custrs)

custrs_row_nans.NaNportions.describe()

Average NaNs per row:  0.3558
Max NaNs per row:  0.8537
Min NaNs per row:  0.0759


count    150000.000000
mean          0.355753
std           0.287805
min           0.075881
25%           0.165312
50%           0.195122
75%           0.747967
max           0.853659
Name: NaNportions, dtype: float64

There is on average a 28% missing values per row in `AZDIAS` and around 35% NaNs per row in `CUSTOMERS`.

In [ ]:
# Save image location
nan_rows_distrb_file = '/content/drive/MyDrive/workspace/arvato_data/nan_rows_distrb_{}.png'.format(ts) \
if IN_COLAB else 'nan_rows_distrb_{}.png'.format(ts)

with sns.axes_style("whitegrid"):
    fig, ax = plt.subplots(1, 2, figsize=(10,5))
    sns.histplot(ax=ax[0], data=azdias_row_nans, x="NaNportions", bins=np.linspace(0, 1, 11)).set(title='[AZDIAS] Row counts per NaN portions')
    sns.histplot(ax=ax[1], data=custrs_row_nans, x="NaNportions", bins=np.linspace(0, 1, 11)).set(title='[CUSTOMERS] Row counts per NaN portions')
    # copyright text
    fig.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
    fig.tight_layout()
    fig.savefig(nan_rows_distrb_file, transparent=False, bbox_inches="tight")
    plt.close(fig)

As seen, both datasets have most rows with missing values in the range 10% - 30% per row.

* `AZDIAS`

In [ ]:
azd_row_nan_bins = azdias_row_nans.NaNportions.value_counts(bins=np.linspace(0, 1, 11), sort=False)

azd_row_nan_bins = azd_row_nan_bins.rename_axis("Bins").reset_index(name="Count")

azd_row_nan_bins["RowPortion"] = azd_row_nan_bins.Count / azdias.shape[0]

azd_row_nan_bins

,Bins,Count,RowPortion
0,"(-0.001, 0.1]",243,0.000810
1,"(0.1, 0.2]",116662,0.388873
2,"(0.2, 0.3]",131302,0.437673
3,"(0.3, 0.4]",15065,0.050217
4,"(0.4, 0.5]",1140,0.003800
5,"(0.5, 0.6]",1695,0.005650
6,"(0.6, 0.7]",1949,0.006497
7,"(0.7, 0.8]",6741,0.022470
8,"(0.8, 0.9]",25203,0.084010
9,"(0.9, 1.0]",0,0.000000


In [ ]:
# 20% NaN counts and dataset portion - AZDIAS
azd_row_nan_bins.loc[:2,].sum()

Count         248207.000000
RowPortion         0.827357
dtype: float64

* `CUSTOMERS`

In [ ]:
cus_row_nan_bins = custrs_row_nans.NaNportions.value_counts(bins=np.linspace(0, 1, 11), sort=False)

cus_row_nan_bins = cus_row_nan_bins.rename_axis("Bins").reset_index(name="Count")

cus_row_nan_bins["RowPortion"] = cus_row_nan_bins.Count / custrs.shape[0]

cus_row_nan_bins

,Bins,Count,RowPortion
0,"(-0.001, 0.1]",250,0.001667
1,"(0.1, 0.2]",80654,0.537693
2,"(0.2, 0.3]",24487,0.163247
3,"(0.3, 0.4]",4342,0.028947
4,"(0.4, 0.5]",234,0.001560
5,"(0.5, 0.6]",260,0.001733
6,"(0.6, 0.7]",833,0.005553
7,"(0.7, 0.8]",2376,0.015840
8,"(0.8, 0.9]",36564,0.243760
9,"(0.9, 1.0]",0,0.000000


In [ ]:
# 20% NaN counts and dataset portion - CUSTOMERS
cus_row_nan_bins.loc[:2,].sum()

Count         105391.000000
RowPortion         0.702607
dtype: float64

####Format data in long-form for plotting with `sns.barplot`. 

In [ ]:
merge_row_nan_bins = pd.merge(azd_row_nan_bins, cus_row_nan_bins,
                              on='Bins', suffixes=("_azd", "_cus"))

rows_nan_portns_melt = merge_row_nan_bins[
                                          ['Bins', 'RowPortion_azd', 'RowPortion_cus']
                                         ].melt(id_vars=('Bins'))

rows_nan_portns_melt.head()

,Bins,variable,value
0,"(-0.001, 0.1]",RowPortion_azd,0.000810
1,"(0.1, 0.2]",RowPortion_azd,0.388873
2,"(0.2, 0.3]",RowPortion_azd,0.437673
3,"(0.3, 0.4]",RowPortion_azd,0.050217
4,"(0.4, 0.5]",RowPortion_azd,0.003800


In [ ]:
# Save image location
nan_rows_perc_file = '/content/drive/MyDrive/workspace/arvato_data/nan_rows_perc_{}.png'.format(ts) \
if IN_COLAB else 'nan_rows_perc_{}.png'.format(ts)

with sns.axes_style("whitegrid"):
    plt.figure(figsize=(8,5))
    sns.barplot(data=rows_nan_portns_melt, x="value", y="Bins", hue="variable", palette="Blues_d").set(title='[AZDIAS, CUSTOMERS] Rows portions per NaN bin')
    # copyright text
    plt.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
    plt.tight_layout()
    plt.savefig(nan_rows_perc_file, transparent=False, bbox_inches="tight")
    plt.close()


In both datasets most rows have NaNs in the range 10% - 30%.


####Column type and exploration for the attributes with values range `'…'`

Finally we have the special cases where the values' pool was `'…'`, described with the `attrs_val_dict_any` dictionary mapping. These include a wide range of numerical values:

* `AZDIAS`

In [ ]:
# Check `attrs_val_dict_any` special cases types:
azdias.loc[:, attrs_val_dict_any.keys()].dtypes

ANZ_HAUSHALTE_AKTIV    float64
ANZ_HH_TITEL           float64
ANZ_PERSONEN           float64
ANZ_TITEL              float64
GEBURTSJAHR            float64
KBA13_ANZAHL_PKW       float64
MIN_GEBAEUDEJAHR       float64
dtype: object

In [ ]:
azdias.loc[:, attrs_val_dict_any.keys()].describe()

,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_PERSONEN,ANZ_TITEL,GEBURTSJAHR,KBA13_ANZAHL_PKW,MIN_GEBAEUDEJAHR
count,266258.000000,8166.000000,263595.000000,1077.000000,167731.000000,264109.000000,268489.000000
mean,8.395676,1.371296,1.800823,1.073352,1967.093161,618.351438,1993.292340
std,15.846538,1.456124,1.117183,0.274717,17.848186,339.685636,3.358749
min,1.000000,1.000000,1.000000,1.000000,1900.000000,1.000000,1985.000000
25%,2.000000,1.000000,1.000000,1.000000,1955.000000,382.000000,1992.000000
50%,4.000000,1.000000,1.000000,1.000000,1967.000000,548.000000,1992.000000
75%,10.000000,1.000000,2.000000,1.000000,1981.000000,776.000000,1993.000000
max,595.000000,23.000000,40.000000,4.000000,2017.000000,2300.000000,2016.000000


We may observe that values for the above attributes look acceptable.

In [ ]:
# Count unique values per attribute (column)
azdias.loc[:, attrs_val_dict_any.keys()].nunique()

ANZ_HAUSHALTE_AKTIV     269
ANZ_HH_TITEL             20
ANZ_PERSONEN             19
ANZ_TITEL                 4
GEBURTSJAHR             115
KBA13_ANZAHL_PKW       1260
MIN_GEBAEUDEJAHR         32
dtype: int64

In [ ]:
# Discrete unique values per column
azdias.loc[:, attrs_val_dict_any.keys()].apply(lambda col: col.unique())

ANZ_HAUSHALTE_AKTIV    [1.0, 4.0, 10.0, 8.0, 2.0, 9.0, 80.0, 11.0, na...
ANZ_HH_TITEL           [nan, 1.0, 3.0, 2.0, 7.0, 8.0, 10.0, 4.0, 13.0...
ANZ_PERSONEN           [3.0, 4.0, 2.0, 1.0, nan, 9.0, 5.0, 6.0, 7.0, ...
ANZ_TITEL                                      [nan, 1.0, 2.0, 3.0, 4.0]
GEBURTSJAHR            [nan, 1946.0, 1988.0, 2005.0, 1985.0, 1961.0, ...
KBA13_ANZAHL_PKW       [539.0, 578.0, 1179.0, 563.0, 507.0, 330.0, 38...
MIN_GEBAEUDEJAHR       [1994.0, 1992.0, 1993.0, 1995.0, nan, 1996.0, ...
dtype: object

* `CUSTOMERS`

In [ ]:
# Check `attrs_val_dict_any` special cases types:
custrs.loc[:, attrs_val_dict_any.keys()].dtypes

ANZ_HAUSHALTE_AKTIV    float64
ANZ_HH_TITEL           float64
ANZ_PERSONEN           float64
ANZ_TITEL              float64
GEBURTSJAHR            float64
KBA13_ANZAHL_PKW       float64
MIN_GEBAEUDEJAHR       float64
dtype: object

In [ ]:
custrs.loc[:, attrs_val_dict_any.keys()].describe()

,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_PERSONEN,ANZ_TITEL,GEBURTSJAHR,KBA13_ANZAHL_PKW,MIN_GEBAEUDEJAHR
count,108992.000000,4779.000000,107914.000000,2170.000000,76998.000000,109830.000000,110885.000000
mean,5.030690,1.528144,2.387429,1.076498,1949.740500,666.902841,1993.052126
std,14.246347,2.065431,1.324896,0.284291,14.909137,340.248578,3.062568
min,1.000000,1.000000,1.000000,1.000000,1900.000000,5.000000,1985.000000
25%,1.000000,1.000000,1.000000,1.000000,1939.000000,430.000000,1992.000000
50%,1.000000,1.000000,2.000000,1.000000,1948.000000,593.000000,1992.000000
75%,4.000000,1.000000,3.000000,1.000000,1960.000000,828.000000,1992.000000
max,395.000000,23.000000,21.000000,5.000000,2017.000000,2300.000000,2016.000000


In [ ]:
# Count unique values per attribute (column)
custrs.loc[:, attrs_val_dict_any.keys()].nunique()

ANZ_HAUSHALTE_AKTIV     209
ANZ_HH_TITEL             19
ANZ_PERSONEN             16
ANZ_TITEL                 4
GEBURTSJAHR             112
KBA13_ANZAHL_PKW       1246
MIN_GEBAEUDEJAHR         32
dtype: int64

In [ ]:
# Discrete unique values per column
custrs.loc[:, attrs_val_dict_any.keys()].apply(lambda col: col.unique())

ANZ_HAUSHALTE_AKTIV    [1.0, nan, 27.0, 7.0, 74.0, 2.0, 9.0, 29.0, 14...
ANZ_HH_TITEL           [nan, 2.0, 1.0, 13.0, 5.0, 4.0, 20.0, 17.0, 9....
ANZ_PERSONEN           [2.0, 3.0, nan, 4.0, 1.0, 5.0, 7.0, 6.0, 8.0, ...
ANZ_TITEL                                      [nan, 1.0, 2.0, 3.0, 5.0]
GEBURTSJAHR            [nan, 1979.0, 1960.0, 1942.0, 1971.0, 1914.0, ...
KBA13_ANZAHL_PKW       [1201.0, nan, 428.0, 513.0, 1300.0, 422.0, 110...
MIN_GEBAEUDEJAHR       [1992.0, nan, 1994.0, 1997.0, 1995.0, 1996.0, ...
dtype: object

## Identifying (if any) columns to drop

At this point we will proceed concatenating the two samples in a single dataframe, creating a new feature `TARGET` that will be `1` for customers and `-1` for unknown.

This will combine attribute characteristics from both datasets and help us make decisions best on both samples.

First let us check if there are any common user IDs in LNR columns:

In [ ]:
np.intersect1d(azdias.LNR.values, custrs.LNR.values)

array([], dtype=int64)

Concatenate the dataframes:

In [ ]:
# Create a new dataframe from `azdias` and `customers` dataframes creating a `target` attribute:
df_train = pd.concat([azdias.assign(TARGET=-1),
                      custrs.drop(columns=["CUSTOMER_GROUP", "ONLINE_PURCHASE", "PRODUCT_GROUP"])
                      .assign(TARGET=1)])

# Shuffle dataframe
df_train = df_train.sample(frac=1).reset_index(drop=True)

# Keep "LNR" IDs
clientIDs = df_train["LNR"]

# Remove "LNR" and "EINGEFUEGT_AM" columns which are client IDs and entry datetimes
df_train.drop(columns=["LNR", "EINGEFUEGT_AM"], inplace=True)

df_train.shape

(450000, 365)

In [ ]:
df_train.dtypes.value_counts()

float64     357
int64         5
category      1
category      1
category      1
dtype: int64

In [ ]:
# Convert float columns to int.
df_train_int_cols = df_train.select_dtypes(include=['float64']).fillna(-1.0).astype(int)
df_train.loc[:, df_train_int_cols.columns] = df_train_int_cols

In [ ]:
df_train.dtypes.value_counts()

int64       362
category      1
category      1
category      1
dtype: int64

### One-hot encoding

One-hot encode categorical attributes `CAMEO_DEU_2015, D19_LETZTER_KAUF_BRANCHE, OST_WEST_KZ`.

In [ ]:
df_train.select_dtypes(include='category').head()

,CAMEO_DEU_2015,D19_LETZTER_KAUF_BRANCHE,OST_WEST_KZ
0,6F,D19_VERSICHERUNGEN,W
1,5F,NaN,O
2,4B,D19_SONSTIGE,W
3,1D,D19_UNBEKANNT,W
4,NaN,NaN,NaN


In [ ]:
df_train = pd.get_dummies(df_train)

df_train.shape

(450000, 443)

irrelevant features, highly correlated features, and missing values can prevent the model from learning and decrease generalization performance on the testing data

## Feature selection to keep only the most useful attributes

* Start with **collinear** attributes
(Ref: https://chrisalbon.com/code/machine_learning/feature_selection/drop_highly_correlated_features/)

Calculate correlation on a stratified sample of the data, since `azdias` dataframe is large enough that notebook runs out of memory while calculating the correlation matrix.

In [ ]:
def get_stratified_samples(df, fracs=np.array([0.2, 0.8])):
    """Split an input df into two stratified subsets of sizes indicated
    by the fracs array.
    """
    # shuffle input df
    df_ = df.sample(frac=1) 
    # split into 2 parts, return df_smpl_a, df_smpl_b
    return np.array_split(df_, (fracs[:-1].cumsum() * len(df_)).astype(int))

# azd_cp_smpl_a, azd_cp_smpl_b = get_stratified_samples(azd_cp)

# azd_cp_smpl_a.shape, azd_cp_smpl_b.shape

In [ ]:
def drop_collinear_cols(df, threshold=0.95):
    """Ιdentifies the highly correlated variables based on the absolute
    magnitude of the Pearson correlation coefficient being greater than threshold (0.9)

    Args:
        df (pd.Dataframe): Input dataframe to work on
        threshold (float): threshold value above which the columns will be selected
    
    Returns:
        np.array of selected columns based on the threshold value.
    """
    # Calculate absolute value correlation matrix from the input df.
    # corr_mat = df.corr().abs()
    corr_mat = np.abs(np.corrcoef(df, rowvar=False))
    # Convert to dataframe
    corr_mat = pd.DataFrame(corr_mat, index=df.columns, columns=df.columns)
    # Select upper triangle of correlation matrix
    upr = corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(np.bool))
    # Find index of attribute columns with correlation greater than `threshold`
    return np.array([c for c in upr.columns if any(upr[c] > threshold)])


In [ ]:
# Find attribute columns with correlation greater than 0.95
to_drop = drop_collinear_cols(df_train)

to_drop, len(to_drop)

(array(['ANZ_STATISTISCHE_HAUSHALTE', 'CAMEO_INTL_2015', 'KBA05_KRSHERST2',
        'KBA05_KRSHERST3', 'KBA05_SEG2', 'KBA13_HALTER_30',
        'KBA13_HALTER_40', 'KBA13_HALTER_55', 'KBA13_HALTER_66',
        'KBA13_HERST_SONST', 'KBA13_KMH_250', 'KBA13_MERCEDES',
        'KBA13_SEG_VAN', 'KBA13_VW', 'LP_FAMILIE_GROB',
        'LP_LEBENSPHASE_GROB', 'LP_STATUS_GROB', 'PLZ8_BAUMAX', 'PLZ8_GBZ',
        'PLZ8_HHZ'], dtype='<U26'), 20)

In [ ]:
df_train.drop(columns=to_drop, inplace=True)

df_train.shape

(450000, 423)

###Search for more columns with high NaN ratio, check their significance.

References:
* https://www.kaggle.com/questions-and-answers/131935
* https://stats.stackexchange.com/questions/221332/variance-of-a-distribution-of-multi-level-categorical-data

Check attributes with NaN ratio more than **0.8**.

In [ ]:
df_train_nans = cols_nans_info(df_train.replace(-1, np.NaN))

df_train_nans

Average NaNs per column:  0.256
Max NaNs per column:  0.9991
Min NaNs per column:  0.0


,Columns,NaNcounts,NaNportions
0,AGER_TYP,306555,0.681233
1,AKT_DAT_KL,61385,0.136411
2,ALTER_HH,158634,0.352520
3,ALTER_KIND1,413670,0.919267
4,ALTER_KIND2,436210,0.969356
...,...,...,...
418,D19_LETZTER_KAUF_BRANCHE_D19_VERSICHERUNGEN,0,0.000000
419,D19_LETZTER_KAUF_BRANCHE_D19_VOLLSORTIMENT,0,0.000000
420,D19_LETZTER_KAUF_BRANCHE_D19_WEIN_FEINKOST,0,0.000000
421,OST_WEST_KZ_O,0,0.000000


In [ ]:
# Select the columns with NaNs more than 80% to check for variability
df_train_nans_gt80 = df_train_nans.loc[(df_train_nans["NaNportions"] > .8), "Columns"].values

df_train_nans_gt80, df_train_nans_gt80.size

(array(['ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4',
        'ANZ_HH_TITEL', 'ANZ_KINDER', 'ANZ_TITEL', 'D19_BANKEN_ANZ_12',
        'D19_BANKEN_ANZ_24', 'D19_BANKEN_DIREKT', 'D19_BANKEN_GROSS',
        'D19_BANKEN_LOKAL', 'D19_BANKEN_ONLINE_QUOTE_12',
        'D19_BANKEN_REST', 'D19_BEKLEIDUNG_GEH', 'D19_BILDUNG',
        'D19_BIO_OEKO', 'D19_DIGIT_SERV', 'D19_DROGERIEARTIKEL',
        'D19_ENERGIE', 'D19_FREIZEIT', 'D19_GARTEN', 'D19_HANDWERK',
        'D19_KINDERARTIKEL', 'D19_KOSMETIK', 'D19_LEBENSMITTEL',
        'D19_NAHRUNGSERGAENZUNG', 'D19_RATGEBER', 'D19_SAMMELARTIKEL',
        'D19_SCHUHE', 'D19_TELKO_ANZ_12', 'D19_TELKO_ANZ_24',
        'D19_TELKO_MOBILE', 'D19_TELKO_ONLINE_QUOTE_12', 'D19_TELKO_REST',
        'D19_TIERARTIKEL', 'D19_VERSAND_REST', 'D19_VERSI_ANZ_12',
        'D19_VERSI_ANZ_24', 'D19_VERSI_ONLINE_QUOTE_12',
        'D19_WEIN_FEINKOST', 'HH_DELTA_FLAG', 'KBA05_ANTG4', 'KBA05_SEG6',
        'KONSUMZELLE', 'SOHO_KZ', 'TITEL_KZ', 'UNGLEICHENN_FLA

In [ ]:
df_train.filter(df_train_nans_gt80).dtypes.value_counts()

int64    48
dtype: int64

We will calculate the **entropy** on the above (ordinal variables) as a measure of the amount of information in a categorical variable. We will attempt to identify columns with little to no information at all.

To do this we will create a function to calculate the entropy in a given column.

In [ ]:
def get_entropy(labels):
    """Calculate the entropy of a data series.

    Calculate the entropy based on Shannon's entropy (log2).
    Scipy entropy() will normalize values' frequencies if
    they don’t sum to 1.

    Args:
        labels (pd.Series): pandas Series to calculate entropy
    
    Returns:
        ent: The calculated entropy of the input labels
    """
    # get values' frequencies
    vals_freq = labels.value_counts(sort=False)
    ent = entropy(vals_freq, base=2)
    
    return ent

A mapping of attribute names (columns) and entropy can be created. This will allow us to decide if we should drop some more columns with a high percentage of NaNs that also offer little to no information.

In [ ]:
def drop_nan_cols(df, df_nans_cols, nan_thres=.8, use_entropy=True, ent_thres=1.0):
    """Get df columns with higher NaN percent than  nan_thres (> 80%) and return
    the ones with entropy < ent_thres. If use_entropy=False return all columns
    with NaN portion higher than nan_thres.

    Returns:
        drop_cols (np.array): Higher percentage NaN columns list (with low entropy)
        ent (dict): Mapping of attributes names with entropy values for selected
        columns range (or None if entropy is not used). 
    """
    # Select the columns with NaNs more than nan_thres to check for variability
    df_nans_cols_gt80 = df_nans_cols.loc[(df_nans_cols["NaNportions"] > nan_thres),
                                         "Columns"].values
    # Filter the dataset to select the above columns only.
    df_var_nan80 = df.filter(df_nans_cols_gt80)
    drop_cols = df_var_nan80.values
    ent = None
    if use_entropy:
        # Calculate entropy & create mapping
        ent = {i: round(get_entropy(df_var_nan80[i]), 4) for i in df_var_nan80}
        # From the dictionary we will select the columns with entropy less than ent_thres
        df_ent_drop = [i for i in ent if ent[i] < ent_thres]
        drop_cols = np.array(df_ent_drop)

    return drop_cols, ent


In [ ]:
# Get df_train columns with higher NaN percent (> 80%) and return the ones with entropy < 1.0
df_train_ent_drop, df_train_var_nan80_ent = drop_nan_cols(df_train, df_train_nans)

df_train_ent_drop, len(df_train_ent_drop)

(array(['ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4',
        'ANZ_HH_TITEL', 'ANZ_KINDER', 'ANZ_TITEL', 'D19_BANKEN_ANZ_12',
        'D19_BANKEN_ANZ_24', 'D19_BANKEN_GROSS', 'D19_BANKEN_LOKAL',
        'D19_BANKEN_ONLINE_QUOTE_12', 'D19_BANKEN_REST',
        'D19_BEKLEIDUNG_GEH', 'D19_BILDUNG', 'D19_BIO_OEKO',
        'D19_DIGIT_SERV', 'D19_DROGERIEARTIKEL', 'D19_ENERGIE',
        'D19_FREIZEIT', 'D19_GARTEN', 'D19_HANDWERK', 'D19_KINDERARTIKEL',
        'D19_KOSMETIK', 'D19_LEBENSMITTEL', 'D19_NAHRUNGSERGAENZUNG',
        'D19_RATGEBER', 'D19_SAMMELARTIKEL', 'D19_SCHUHE',
        'D19_TELKO_ANZ_12', 'D19_TELKO_ANZ_24', 'D19_TELKO_MOBILE',
        'D19_TELKO_ONLINE_QUOTE_12', 'D19_TELKO_REST', 'D19_TIERARTIKEL',
        'D19_VERSI_ANZ_12', 'D19_VERSI_ANZ_24',
        'D19_VERSI_ONLINE_QUOTE_12', 'D19_WEIN_FEINKOST', 'HH_DELTA_FLAG',
        'KBA05_ANTG4', 'KBA05_SEG6', 'KONSUMZELLE', 'SOHO_KZ', 'TITEL_KZ',
        'UNGLEICHENN_FLAG'], dtype='<U26'), 46)

In [ ]:
# Create plot data, in a new df get 'Attributes', 'Entropy' and "NaNportions"
plt_data = pd.DataFrame(df_train_var_nan80_ent.items(), columns=['Attributes', 'Entropy'])

# Get NaN portions for the attributes with NaNs more than 80%
plt_data["NaNportions"] = df_train_nans.loc[df_train_nans["Columns"]
                                            .isin(df_train_nans_gt80),"NaNportions"].values

# Sort by `Entropy` values
plt_data.sort_values("Entropy", ascending=False, inplace=True)

plt_data.head()

,Attributes,Entropy,NaNportions
36,D19_VERSAND_REST,1.0198,0.829644
9,D19_BANKEN_DIREKT,1.0002,0.835387
23,D19_KINDERARTIKEL,0.9998,0.828273
18,D19_DROGERIEARTIKEL,0.9767,0.849622
32,D19_TELKO_MOBILE,0.9498,0.821413


In [ ]:
# Save image location
nan_perc_entr_file = f'{colab_dir if IN_COLAB else remote_dir}nan_perc_entr_{ts}.png'

# Let's visualize the results
with sns.axes_style("whitegrid"), sns.plotting_context("paper", rc={"font.size": 8,}):
    fig, ax = plt.subplots(1, 2, figsize=(12, 7), sharey=True)
    sns.barplot(ax=ax[0], data=plt_data, x='Entropy', y='Attributes', color='royalblue').set(
        title='[AZDIAS] Entropy for high percentage (> 0.8) NaN attributes')
    sns.barplot(ax=ax[1], data=plt_data, x='NaNportions', y='Attributes', color='cornflowerblue').set(
        title='[AZDIAS] Attributes NaN portions (0.8 min)', xlim=(0.8,1.0))
    # copyright text
    fig.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
    fig.tight_layout()
    fig.savefig(nan_perc_entr_file, transparent=False, bbox_inches="tight")
    plt.close(fig)

Plots show that some of the attributes found with the lowest entropy also have a high number of NaNs. Checking entropy as a measure of information per column will allow us to drop those columns with a high percentage of NaNs, with more confidence than just choosing to drop columns above a high NaN percentage threshold.

###Drop previously identified attributes

* Drop collinear attributes in `to_drop` array.
* Drop attributes in `azd_ent_drop` with NaNs more than 80%, also identified as having the lowest entropy among the rest with such a high NaN percentage.
* Drop `CUSTOMERS` specific attributes (`'CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'`).
* Drop the datetime `EINGEFUEGT_AM`.

In [ ]:
df_train.drop(columns=df_train_ent_drop, inplace=True)

df_train.shape

(450000, 377)

At this point all the previous steps can be aggregated in a single function to help us manipulate datasets in the following steps. We will include `zero_imp_attrs` as an optional parameter that will be used in the following analysis (in parts 2 and 3).

In [ ]:
def wrangle_df(df, val_pools, common_attrs, replace_nan, zero_imp_attrs=None):
    """Clean and fix an input demographics dataframe for this project.

    This function is for cleaning and fixing datasets for this project. It
    returns a clean dataframe with several columns dropped based on identified
    NaNs, entropy and  collinearity. Additionally, values per column are checked
    and replaced with NaNs if they fall outside an accepted values pool.

    Args:
        df (pd.Dataframe): Input demographics datafrme (The notebook crashes when
            the entire `azdias` is loaded).
        val_pools (dict): A dictionary with keys the attributes names and values
            lists with the allowed values for each attribute. e.g.:
            {'AGER_TYP': [nan, -1, 0, 1, 2, 3], ...}
        common_attrs (np.array): Common attributes (attrs_common) between
            demographic dataset and excel files. e.g.:
            (array(['AKT_DAT_KL', 'ALTERSKATEGORIE_FEIN', ...)]
        replace_nan (dict): Mapping of attributes and values to replace with NaNs
            within each attribute. e.g. {'AGER_TYP': 9} will replace 9 in 'AGER_TYP'
            with NaN.
        zero_imp_attrs (array like or None): `zero_val_attrs` computed zero
            importance attributes. If provided these will be dropped from the
            input df.
    
    Returns:
        df (pd.Dataframe): The cleaned and fixed df.
    """
    # Keep "LNR" IDs
    clientIDs = df["LNR"]

    # Drop `customer` specific attributes.
    other_to_drop = np.array(["CUSTOMER_GROUP", "ONLINE_PURCHASE", "PRODUCT_GROUP"])

    # Remove "LNR" and "EINGEFUEGT_AM" columns which are client IDs and entry datetimes
    df.drop(columns=np.union1d(["LNR", "EINGEFUEGT_AM"], other_to_drop),
            errors='ignore', inplace=True)

    # find object columns
    cat_cols = df.select_dtypes(include=object).columns.tolist()
    # Convert to category columns
    df[cat_cols] = df[cat_cols].astype('category')

    # Replace unknown and specific values with NaNs
    # for every key (attribute) in the dictionary created earlier
    for k in val_pools:
        # if the attribute is among the common ones
        if k in common_attrs:
            # select all those rows in the column that fall outside the attritbute's specified values pool and replace them with NaN.
            df.loc[~df[k].isin(val_pools[k]), k] = np.NaN
    # Use a dictionary mapping of attributes to replace value 9 (unknown) for NaN for specific attributes.
    df.replace(replace_nan, np.NaN, inplace=True)

    # NaNs
    # Total
    print("Total df NaNs portion: ", round(df.isna().sum().sum() / df.size, 4))
    # Columns
    print("Column NaNs:")
    df_cols_nans = cols_nans_info(df)
    # Rows
    print("Row NaNs:")
    df_rows_nans = rows_nans_info(df)

    # Convert float columns to int.
    df_int_cols = df.select_dtypes(include=['float64']).fillna(-1.0).astype(int)
    df.loc[:, df_int_cols.columns] = df_int_cols

    # One-hot encoding, for ['CAMEO_DEU_2015', 'D19_LETZTER_KAUF_BRANCHE', 'OST_WEST_KZ']
    df = pd.get_dummies(df)

    # Find attribute columns with correlation greater than 0.95 and drop them
    corr_cols_to_drop = drop_collinear_cols(df)
    df.drop(columns=corr_cols_to_drop, inplace=True)

    # Select the columns with NaNs more than 80% and with entropy less than 1.0.
    df_ent_drop, ent = drop_nan_cols(df, df_cols_nans)
    
    # Drop above selected NaN attributes and supplied zero importance attributes.
    zero_imp_drop = zero_imp_attrs or []
    df.drop(columns=np.union1d(df_ent_drop, zero_imp_drop),
            errors='ignore', inplace=True)

    return df.assign(LNR=clientIDs.values)

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

> The goal of unsupervised learning is to find the underlying structure of a dataset, group that data according to similarities, and represent that dataset in a compressed format.

The unsupervised learning algorithm can be further categorized into two types of problems:

* **Clustering**: Clustering is a method of grouping the objects into clusters such that objects with most similarities remain in a group and have less or no similarities with the objects of another group. Cluster analysis finds the commonalities between the data objects and categorizes them as per the presence and absence of those commonalities.

* **Association**: An association rule is an unsupervised learning method which is used for finding the relationships between variables in the large database. It determines the set of items that occurs together in the dataset. Association rule makes marketing strategy more effective. For example, people who buy X item (suppose a bread) also tend to purchase Y (Butter/Jam) item. A typical example of Association rule is Market Basket Analysis.

Ref: https://www.javatpoint.com/unsupervised-machine-learning

Popular unsupervised learning algorithms:

* K-means clustering
* KNN (k-nearest neighbors)
* Hierarchical clustering
* Anomaly detection
* Neural Networks
* Principal Component Analysis
* Independent Component Analysis
* Apriori algorithm
* Singular value decomposition

### Clustering with Mini Batch K-Means

We will choose the KMeans algorithm for data clustering, because it is a general-purpose clustering algorithm with even cluster sizes and has good scalability for very large samples. We will specifically use the [MiniBatch code](https://scikit-learn.org/stable/modules/clustering.html#mini-batch-kmeans), a variant of the KMeans algorithm which uses mini-batches to reduce the computation time.

The K-means algorithm aims to choose centroids that minimise the inertia, or within-cluster sum-of-squares criterion. However, inertia is not a normalized metric, becoming a drawback for the use of KMeans. To overcome this it is usual practice to run a dimensionality reduction algorithm such as Principal component analysis (PCA) prior to k-means clustering. K-means algorithm is very much susceptible to the range in which features are measured, we will therefore normalize our data in the process.

PCA can be performed using the correlation matrix or the covariance matrix of the data. These will provide different results. In general the covariance matrix is used when the variable scales are similar and the correlation matrix when variables are on different scales ([Ref.](https://stats.stackexchange.com/a/78/336933)).

For this data we would like to have a more informative PCA so we will go with  PCA on correlation, since the covariance matrix on our data will be primarily dominated by the amount of missing values. 

A pipeline that will initially impute missing values with `most_frequent` values, then standardize the data with `MinMaxScaler` as we cannot assume that the shape of all your features follows a normal distribution, and finally fit PCA will allow us to:

* Initially fit the PCA with all components to acquire the full picture of  variability in the data.
* Examine the `explained_variance_ratio_` and decide on the number of components to finally use. Typically, we want the explained variance to be at least 95%.
* Fit PCA with the selected components (`n_components = .95`).

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

scaler = MinMaxScaler()

#### Impute missing values with ***random selection***

> The  Random  Selection  approach  is  a  method  based  on randomly assigning  a value to  the missing data.  The values randomly selected  are framed  in a specific  range of  values, which should  have the same  characteristics as the  values in the  variable  with  the  missingness (numerical  or  categorical data).  Each  number  in  between  the  range  has  the  same probability of being assigned to the missing data.



Ref: https://www.researchgate.net/publication/326435546_Missing_Data_Imputation_for_Ordinal_Data

We will create a function to apply to the dataframe for imputing missing values with random selection from the column's values range.

In [ ]:
# Initialize random generator
rng = np.random.default_rng()

def imput_nan_rand(col, in_place=False):
    """Get a pd.Series with NaNs and impute missing values with random selection
    from the column's values range.
    """
    c = col if in_place else col.copy()
    # get indices of `-1` occurrences
    idxs = np.where(c == -1)[0]
    # initialize boolean mask
    m = np.ones(c.size, dtype=bool)
    # change boolean mask at indices
    m[idxs] = False
    # create column unique values range
    pool = np.unique(c[m]).astype(int)
    # assign random values from the pool at `-1` locations
    c[~m] = rng.choice(pool, size=idxs.size, shuffle=False)
    return c.astype(int)


In [ ]:
# Create train and target sets.
train = df_train.drop(columns=["TARGET"])
target = df_train["TARGET"]

# Impute missing values with Random Selection
norm_train = train.apply(imput_nan_rand)

# Normalization
norm_train = pd.DataFrame(scaler.fit_transform(norm_train))
norm_train.columns = train.columns
norm_train.index = train.index

A view of our original cleaned data. Let's produce a scatter matrix for some pairs of attributes in the data to have a view of data before imputing, scaling.

In [ ]:
# Save image location
pairplot1_file = f'{colab_dir if IN_COLAB else remote_dir}pairplot_orig_clean_{ts}.png'

pr1plt = sns.pairplot(train.sample(100000).iloc[:, :4], diag_kind='kde')
pr1plt.fig.suptitle("Sample data view BEFORE imputing, scaling", y=1.02)
# copyright text
pr1plt.fig.text(.4 ,-.03, cprt, bbox=bb_cprt_args)
plt.savefig(pairplot1_file, transparent=False, bbox_inches="tight")
plt.close()

A view of the scaled data **after** imputing missing values with random selection and normalization.

In [ ]:
# Save image location
pairplot2_file = f'{colab_dir if IN_COLAB else remote_dir}pairplot_imputd_scld_{ts}.png'

pr2plt = sns.pairplot(norm_train.sample(100000).iloc[:, :4], diag_kind='kde')
pr2plt.fig.suptitle("Sample data view AFTER imputing, scaling", y=1.02)
# copyright text
pr2plt.fig.text(.4 ,-.03, cprt, bbox=bb_cprt_args)
plt.savefig(pairplot2_file, transparent=False, bbox_inches="tight")
plt.close()

In [ ]:
# Initialize PCA model
pca = PCA()

# Fit the model with train data
pca.fit(norm_train.to_numpy())

# components
pca.n_components_

376

In [ ]:
# Let's visualize the results
def plt_pca(pca_model, n_pcs=5, cumul_expl_var=.95, fig_size=(10, 5), save_file=None):
    """Plots first n_pcs principal components and the cumulative variance of
    components.Plots cumulative variance ratio curve marks the cumulative
    explained threshold if < 1.0 indicating the number of PCs required.
    
    Args:
        pca_model: A fitted PCA model (sklearn.decomposition.PCA).
        n_pcs (int): Number of PCs to plot.
        cumul_expl_var (float): Plot information about cumulative variance ratio
        below `cumul_expl_var` threshold.
        fig_size (tuple of int): Figure size for the combined plot.
        
    Returns:
        df (pd.DataFrame): A dataframe ordered by PCs variance ratios with a
            cumulative variance ratio column (sums to 1).
        thres_num_pcs: Number of PCs for `cumul_expl_var` threshold
    """
    pca_comp = n_pcs if pca_model.n_components_ >= 25 else pca_model.n_components_
    df = pd.DataFrame({'PC': np.arange(1, pca_model.n_components_ + 1),
                           'var': pca_model.explained_variance_ratio_})
    df["cumul_var"] = df["var"].cumsum()
    thres_num_pcs = df[df["cumul_var"] <= \
                           cumul_expl_var]["cumul_var"].count()
    if cumul_expl_var < 1.0:
        thres_num_pcs += 1
    with sns.axes_style("whitegrid"), sns.plotting_context("paper"):
        fig, ax = plt.subplots(1, 2, figsize=fig_size)
        # Bar plot of explained variance per PC
        sns.barplot(ax=ax[0], data=df.iloc[:pca_comp, :],
                        x='PC', y='var', color='royalblue')
        ax[0].set(title='Variance explained, first ' + str(pca_comp) + ' PCs',
                xlabel='Principal components')
        # Cumulative explained variance plot
        sns.lineplot(ax=ax[1], data=df['var'].cumsum(), color='r', linewidth=3)
        ax[1].set(title='Scree plot, mark: ' \
                    + str(cumul_expl_var) \
                    + ', ' + str(thres_num_pcs) + ' PCs',
                xlabel='Number of components',
                ylabel='Cumulative explained variance')
        # if the number of PCs at cumul_expl_var is less than the
        # total computed PCs mark it on the chart
        if thres_num_pcs < df.shape[0]:
            # horizontal line marker
            ax[1].axhline(cumul_expl_var, linestyle='--',
                          color='royalblue', lw=2, alpha=0.6)
            # vertical line marker
            ax[1].axvline(thres_num_pcs, linestyle='--',
                          color='royalblue', lw=2, alpha=0.6)
        # copyright text
        fig.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
        fig.tight_layout()
        if save_file:
            fig.savefig(save_file, transparent=False, bbox_inches="tight")
            plt.close(fig)
    return df, thres_num_pcs


In [ ]:
pca_scree_file = f'{colab_dir if IN_COLAB else remote_dir}pca_screeplt_{ts}.png'

pca_df, pcs_num = plt_pca(pca, n_pcs=25, cumul_expl_var=.7, save_file=pca_scree_file)

pca_df.head()

,PC,var,cumul_var
0,1,0.068443,0.068443
1,2,0.044690,0.113133
2,3,0.035743,0.148876
3,4,0.032697,0.181572
4,5,0.023131,0.204703


So, for approximately **0.7** cumulative explained variance ratio line marked components in the figure are required (amount of variance that needs to be explained is greater than the percentage specified by `n_components`).

We will fit PCA again using the number of components for 0.7 cumulative explained variance ratio:

In [ ]:
# Initialize PCA with the new parameter (could also use n_components=.75)
pca = PCA(n_components=pcs_num)

# Fit the model with the new parameter again and apply the dimensionality reduction.
train_reduced = pca.fit_transform(norm_train.to_numpy())

train_reduced.shape

(450000, 104)

In [ ]:
pca.components_.shape

(104, 376)

We have `n` `components_` representing the `p` principal axes in feature space. Each component is an array of loadings (weights) defining the direction of the principal component along which data varies the most. It results in a line in the `p` dimensional feature space.

We can see how each attribute is represneted by these weights by creating a mapping:

In [ ]:
# Create the column names for each PC
pc_names = ['PC' + str(i + 1) for i in range(pca.n_components_)]

# Create a reduced attributes dataframe for later use
train_reduced = pd.DataFrame(train_reduced, columns=pc_names, index=norm_train.index)

# Create loadings - attributes mapping
pca_wghts_df = pd.DataFrame(pca.components_,
                            columns=df_train.drop(columns=["TARGET"]).columns.tolist(),
                            index=pc_names)

pca_wghts_df.shape

(104, 376)

If we transpose this we get:

In [ ]:
pca_wghts_df.T.head(2)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40,...,PC65,PC66,PC67,PC68,PC69,PC70,PC71,PC72,PC73,PC74,PC75,PC76,PC77,PC78,PC79,PC80,PC81,PC82,PC83,PC84,PC85,PC86,PC87,PC88,PC89,PC90,PC91,PC92,PC93,PC94,PC95,PC96,PC97,PC98,PC99,PC100,PC101,PC102,PC103,PC104
AGER_TYP,0.040499,0.019273,0.017980,-0.004219,-0.001565,-0.000258,0.006022,-0.014194,0.005669,-0.010191,0.007729,-0.039241,0.042344,-0.014783,0.019302,0.031102,0.032487,-0.002453,0.020443,0.007879,-0.019790,0.032408,0.054422,-0.150948,-0.049295,-0.079318,-0.030557,-0.173139,0.047624,-0.002524,-0.082730,0.019034,0.106739,0.145640,-0.446251,0.221433,0.214156,0.371410,-0.151337,0.242150,...,-0.008568,0.020501,-0.019924,0.012193,-0.015130,-0.003970,-0.003982,0.005635,0.041221,0.012159,0.000564,-0.013774,0.013212,0.026896,-0.017190,0.027402,0.016086,-0.010980,-0.006706,0.017079,-0.001329,0.014007,0.007533,-0.013356,0.002867,0.007722,-0.038068,-0.017381,0.004318,-0.000098,-0.017653,-0.005291,-0.004985,-0.002139,-0.015422,-0.018455,0.006800,0.014299,0.003058,-0.021616
AKT_DAT_KL,0.152465,0.010389,-0.014757,-0.072644,-0.052135,-0.101453,0.071686,-0.016238,0.005246,0.012983,0.028318,-0.015893,0.020779,0.006200,-0.020310,0.021642,-0.023154,-0.022461,0.074776,0.047494,-0.002094,-0.039537,-0.013757,0.030089,0.045386,-0.074345,0.079225,0.048936,-0.035783,0.023037,-0.017126,-0.020888,-0.171201,-0.017245,0.153162,-0.174816,0.176846,-0.008283,-0.038711,0.000405,...,-0.068520,-0.094211,0.067700,0.080490,0.049474,-0.137913,-0.074241,0.139672,0.056110,0.089608,0.036279,0.067461,0.046748,-0.156053,-0.000363,-0.150017,0.050865,-0.042738,-0.014365,-0.005070,0.174441,0.115032,0.040947,-0.011871,0.029424,0.010613,0.042750,-0.019973,0.020907,-0.044007,0.019236,0.034034,-0.063241,-0.014246,-0.010589,-0.062912,0.034017,-0.125696,-0.079933,0.058497


In [ ]:
# Save image location
pc1pc2_makeup_file = f'{colab_dir if IN_COLAB else remote_dir}PC1PC2_attr_makeup_{ts}.png'

# Let's visualize the results sampling some attributes
plt_attrs_wgts = pca_wghts_df.T[["PC1", "PC2"]].sample(30).rename_axis("Attributes").reset_index()
# Combine the two PCs to create a type of category for plotting purposes.
plt_attrs_wgts = plt_attrs_wgts.melt(id_vars='Attributes')
# Sort by values
plt_attrs_wgts = plt_attrs_wgts.sort_values(by=['value'], ascending=False)
# Plot
with sns.axes_style("whitegrid"), sns.plotting_context("paper", rc={"font.size": 8,}):
    plt.figure(figsize=(10,6))
    sns.barplot(data=plt_attrs_wgts, x='value', y='Attributes', hue='variable',
                palette='Blues_d').set(
                    title="Attributes' influences on first two PCs (sample of 30 attributes)",
                    xlabel="PCs weights")
    # copyright text
    plt.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
    plt.tight_layout()
    plt.savefig(pc1pc2_makeup_file, transparent=False, bbox_inches="tight")
    plt.close()

We would like to examine the makeup of each PCA component based on the weightings of the original attributes that are included in each component. To do so will create a function to help us visually identify top `n` and bottom `n` attributes' influence on each principal component.

In [ ]:
def plot_pc_attrs(df_wghts, pc_n, plot_n, fig_size=(8, 4), save_name=None):
    """Plot top `n` and bottom `n` attributes' influence on each principal component.

    Args:
        df_wghts (pd.DataFrame): Contains mapping of PCs' weights for each attribute
        pc_n (int): Number of PCs to plot.
        plot_n (int): Number of attributes to plot (will be used for top & bottom)
        fig_size (tuple): Size of figure, e.g. (12, 8)
        save_name (str or None): File name to save the figure to.
            If `None` figure will not be saved.
    """
    pc = 'PC' + str(pc_n)
    pc_attrs_wgts = df_wghts.T[pc].sort_values(ascending=False)\
                    .rename_axis("Attributes").reset_index()
    # Plot top n and bottom n
    with sns.axes_style("whitegrid"), sns.plotting_context("paper", rc={"font.size": 8,}):
        plt.figure(figsize=fig_size)
        sns.barplot(data=pc_attrs_wgts.iloc[np.r_[0:plot_n, -plot_n:0]], x=pc,
                    y='Attributes', color='royalblue').set(
                        title="Attributes' influences on {} (top & bottom {})"\
                        .format(pc, plot_n),
                        xlabel="{} weights".format(pc))
        # copyright text
        plt.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
        plt.tight_layout()
        if save_name:
            plt.savefig(save_name, transparent=False, bbox_inches="tight")
            plt.close()

In [ ]:
# Save image location
pc1_makeup_file = f'{colab_dir if IN_COLAB else remote_dir}PC1_attr_makeup_{ts}.png'

plot_pc_attrs(pca_wghts_df, 1, 10, save_name=pc1_makeup_file)

In [ ]:
# Save image location 'PC2'
pc2_makeup_file = f'{colab_dir if IN_COLAB else remote_dir}PC2_attr_makeup_{ts}.png'

plot_pc_attrs(pca_wghts_df, 2, 10, save_name=pc2_makeup_file)

We can create a list of the **two** highest influencing attribute names in each component.

In [ ]:
# initialize list to hold attribute names
pca_list = []
# iterate through the PC columns, sort values and get the top two attributes with
# higher weights. Join names in a single string representing each component.
for i in pca_wghts_df.T:
    pca_list.append('/'.join(list(pca_wghts_df.T[i].sort_values(ascending=False).index[:2])))

pca_list[:4], len(pca_list)

(['CJT_TYP_2/FINANZ_SPARER',
  'FINANZ_ANLEGER/FINANZ_UNAUFFAELLIGER',
  'OST_WEST_KZ_W/STRUKTURTYP',
  'HH_EINKOMMEN_SCORE/OST_WEST_KZ_O'],
 104)

We can now create a dataframe where each individual (row) is described by the number of principal components that we chose previously. Each of these selected components is a linear combination of the original attribute space. We can interpret each of these selected principal components by analyzing the makeup of the component as shown earlier.

In [ ]:
population_transformed = train_reduced.copy()
population_transformed.columns=pca_list
population_transformed.head()

,CJT_TYP_2/FINANZ_SPARER,FINANZ_ANLEGER/FINANZ_UNAUFFAELLIGER,OST_WEST_KZ_W/STRUKTURTYP,HH_EINKOMMEN_SCORE/OST_WEST_KZ_O,ANREDE_KZ/SEMIO_KAEM,KBA13_BAUMAX/LP_FAMILIE_FEIN,OST_WEST_KZ_O/KBA13_KW_61_120,PLZ8_ANTG4/KBA13_ANTG4,KBA13_CCM_3001/KBA13_CCM_2501,KBA05_MAXSEG/KBA05_MOTOR,KBA13_ANTG4/PLZ8_ANTG4,KBA05_MAXBJ/SHOPPER_TYP,VERS_TYP/HEALTH_TYP,KBA05_MOD8/KBA05_SEG9,LP_FAMILIE_FEIN/LP_LEBENSPHASE_FEIN,KBA05_KW3/KBA05_CCM4,KBA13_ALTERHALTER_45/KBA13_HALTER_35,KBA13_VORB_1_2/KBA13_ALTERHALTER_61,D19_GESAMT_ANZ_24/D19_VERSAND_ANZ_24,D19_GESAMT_ANZ_24/D19_VERSAND_ANZ_24,KBA13_KRSZUL_NEU/KBA05_MOD8,KBA05_ANHANG/KBA05_ANTG3,KBA05_ANHANG/KBA05_BAUMAX,SHOPPER_TYP/KBA05_ANTG1,SHOPPER_TYP/KBA13_KRSSEG_VAN,KBA05_ANHANG/KBA05_MOTRAD,KBA05_MOTRAD/BALLRAUM,VHN/D19_SOZIALES,PLZ8_ANTG4/KBA05_ANHANG,PLZ8_ANTG4/KBA05_ANTG3,SHOPPER_TYP/KBA13_ANTG4,PLZ8_ANTG4/KBA13_KRSSEG_VAN,D19_LETZTER_KAUF_BRANCHE_D19_UNBEKANNT/D19_KONSUMTYP,KBA05_ANHANG/KBA05_SEG7,KBA05_MOTRAD/KBA05_ANTG3,KBA05_MOTRAD/AGER_TYP,VHN/KBA05_MOTRAD,AGER_TYP/KBA05_ZUL2,KBA05_SEG8/KBA05_SEG7,KBA05_SEG7/AGER_TYP,...,D19_BANKEN_DIREKT/D19_REISEN,D19_REISEN/KBA13_CCM_1500,D19_HAUS_DEKO/D19_BEKLEIDUNG_REST,D19_BEKLEIDUNG_REST/D19_BANKEN_DIREKT,KBA13_KW_70/KBA05_ANTG2,D19_BEKLEIDUNG_REST/D19_HAUS_DEKO,D19_VERSAND_REST/D19_BEKLEIDUNG_REST,KBA05_MOD1/KBA05_SEG10,KK_KUNDENTYP/D19_VERSAND_ANZ_24,KBA05_ANTG2/D19_VOLLSORTIMENT,KBA05_SEG9/D19_TECHNIK,KBA05_ALTER2/D19_VERSAND_ANZ_12,D19_SONSTIGE/D19_VERSAND_ANZ_12,KBA05_SEG9/KBA05_ALTER3,D19_VOLLSORTIMENT/KBA05_HERST5,KBA05_MOD8/D19_REISEN,D19_BUCH_CD/KBA05_KW3,D19_BUCH_CD/KBA05_MOD8,D19_BUCH_CD/KBA05_KRSVAN,D19_VOLLSORTIMENT/D19_GESAMT_ONLINE_QUOTE_12,RELAT_AB/KBA05_KW3,D19_SONSTIGE/EXTSEL992,KBA05_CCM4/D19_SONSTIGE,KBA05_DIESEL/UMFELD_JUNG,KBA05_DIESEL/RT_UEBERGROESSE,KBA13_VORB_2/KBA13_VORB_1_2,GFK_URLAUBERTYP/KBA05_KRSVAN,D19_SONSTIGE/D19_VERSAND_ANZ_24,KBA05_SEG5/KBA13_SEG_OBERKLASSE,KBA05_BAUMAX/UMFELD_ALT,KBA13_KW_90/KBA13_CCM_3001,KBA05_DIESEL/KBA05_HERST4,KBA13_SEG_OBERKLASSE/KBA13_KRSZUL_NEU,KBA05_MOD4/KBA05_SEG5,KBA13_BJ_2008/RETOURTYP_BK_S,KBA05_VORB2/EXTSEL992,KBA05_HERST4/KBA05_FRAU,D19_VERSAND_ONLINE_QUOTE_12/KBA05_KRSVAN,GFK_URLAUBERTYP/D19_VERSAND_ONLINE_QUOTE_12,KK_KUNDENTYP/D19_GESAMT_ANZ_12
0,-1.234629,-0.867582,-0.578983,-0.220230,-0.742176,0.211376,-0.222328,0.117435,0.625594,-0.276393,0.209671,1.080525,0.073092,-0.005344,-0.725521,0.534920,-0.816195,0.000018,-0.548220,-0.422507,0.358123,0.910408,-0.577757,-0.334013,-0.317721,-0.865876,-0.034087,0.055927,0.069684,0.000181,0.192486,0.326819,-0.207724,0.509368,-0.385418,0.339499,0.397615,0.350358,-0.024274,0.040678,...,-0.127198,0.040113,0.447853,0.491691,0.887629,0.179989,0.231284,0.223842,0.026079,-0.465061,-0.227642,-0.249795,-0.011948,0.248586,-0.388448,0.285329,-0.139759,-0.337783,-0.083996,-0.233873,-0.179353,-0.448052,-0.513697,-0.347848,-0.781476,0.001404,-0.283709,-0.331719,0.221276,-0.163244,0.291595,0.186463,-0.005497,-0.228021,0.307563,-0.713716,0.401752,0.477682,-0.182475,-0.062534
1,0.977389,0.134613,0.048990,-0.160303,-0.804427,-0.352177,0.670749,-1.172047,-1.055501,0.573792,0.661885,-1.608553,0.358878,0.913354,-1.173424,0.805668,-0.148719,0.882287,0.335386,0.451623,0.064425,1.197852,-0.666765,0.302235,0.656651,0.195785,-0.440172,0.129148,-0.404084,0.013804,-0.379258,0.256196,-0.816625,0.160029,0.401891,-0.057911,0.288800,0.054266,0.203565,-0.225481,...,-0.006713,-0.835488,-0.042651,0.597577,0.099311,0.298713,0.140128,0.083948,-0.230444,0.391376,0.510545,0.052131,0.096806,-0.743018,0.162313,-0.162888,-0.008984,0.063964,-0.152078,-0.096144,-0.237473,0.091904,0.133245,0.210970,-0.439297,0.643481,0.427124,0.061224,0.371090,-0.009132,-0.335295,-0.327904,0.320068,-0.003117,-0.307557,-0.310712,-0.153566,0.351870,-0.000796,0.130907
2,0.640617,0.397748,0.081203,-0.250016,0.403747,-1.531851,-0.125694,0.206615,1.062772,-0.607811,-0.043918,-0.782834,0.357914,0.292886,-0.547790,-0.100105,-0.214742,0.114630,-0.347420,-0.215213,0.445487,-0.195424,-0.093167,-0.405749,0.601990,

Feature selection with PCA has left us with a transformed dataset `train` with less dimensions than before. We can proceed to clustering with this dataframe.

In [ ]:
print("Elapsed time: ", str(timedelta(seconds=(timer() - start_run))))

Elapsed time:  0:04:30.844065


##Segmentation using unsupervised clustering

#### Evaluating Clustering Performance - Clusters choice

Our approach for determining the quality of the cluster assignments will be:

* Randomly select k centroids, (k is equal to the number of clusters chosen)
* Record computed sum of the squared error (**SSE**), defined as the sum of the squared Euclidean distances of each point to its closest centroid. (visual **elbow method**). Objective is to minimize SSE.
* Record the **silhouette coefficient**, a measure of cluster cohesion and separation. The silhouette ranges from −1 to +1. A high number, close to 1, indicates that a sample is closer to its cluster than to other clusters. (In scikit-learn [`silhouette_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html) returns the mean Silhouette Coefficient over all samples in a cluster and is only defined if the number of labels (clusters) is greater than 2.)


In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import MiniBatchKMeans

#### Compute SSE and Silhouette coefficient

Using the `MiniBatchKMeans` which is much faster compared to `KMeans` and a `sample_size = 10000` for `silhouette_score` we can achieve acceptable results relatively fast for these large datasets.

In [ ]:
# Create a range of values to try for cluster numbers
n_clusters_range = np.arange(2, 15)

all_Kscores = []
start = timer()
# Run several k-means, incrementing k with each iteration, record the SSE and
# silhouette_score
train_vals = train_reduced.to_numpy()
for k in n_clusters_range:
    #Initialize k-means using the faster MiniBatchKMeans
    clusterer = MiniBatchKMeans(n_clusters=k, random_state=0)
    # Fit standardized data with reduced features
    clusterer.fit(train_vals)
    # Get computed scores, and save them for further reference.
    k_scores = {
        "Clusters": k,
        "SSE": clusterer.inertia_,
        "Sil_coef": silhouette_score(train_vals, clusterer.labels_,
                                     sample_size=10000,
                                     n_jobs=-1),
    }
    all_Kscores.append(k_scores)

print("Time elapsed: ", str(timedelta(seconds=(timer() - start))))

#Create a dataframe of clustering scores that we can use for plotting.
all_Kscores = pd.DataFrame(all_Kscores)
all_Kscores.set_index("Clusters", inplace=True)

all_Kscores.head()

Time elapsed:  0:01:33.096290


,SSE,Sil_coef
Clusters,,
2,8.814419e+06,0.070148
3,8.510458e+06,0.055318
4,8.330091e+06,0.056230
5,8.006994e+06,0.054746
6,7.869831e+06,0.053762


#### Elbow method

Determine the elbow point in the SSE curve using the [kneed](https://github.com/arvkevi/kneed) package:

In [ ]:
# Elbow method knee locator
from kneed import KneeLocator

kl = KneeLocator(n_clusters_range, all_Kscores.SSE.to_list(),
                 curve="convex", direction="decreasing")

kl.elbow

8

Optimal number of clusters is found using the **elbow method**, at the iteration where the SSE curve starts to bend, known as the elbow point at `kl.elbow`.

In [ ]:
# Save image location
elbow_sil_file = f'{colab_dir if IN_COLAB else remote_dir}Elbow_silcoef_{ts}.png'

with sns.axes_style("whitegrid"), sns.plotting_context("paper"):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    # plot elbow
    sns.lineplot(ax=ax[0], x='Clusters', y='SSE', data=all_Kscores, linewidth=3).set(
        xticks=n_clusters_range,
        title="Elbow method for optimal n_clusters")
    ax[0].axvline(kl.elbow, linestyle='--', color='r', lw=1, alpha=0.65)
    # Plot average silhouette
    sns.lineplot(ax=ax[1], x='Clusters', y='Sil_coef', data=all_Kscores, linewidth=3).set(
        xticks=n_clusters_range,
        ylabel='Average silhouette scores',
        title="Silhouette scores for n_clusters")
    # copyright text
    fig.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
    fig.savefig(elbow_sil_file, transparent=False, bbox_inches="tight")
    fig.tight_layout()
    plt.close(fig)

Number of clusters will be chosen from the plot of SSE scores using the elbow method as the decision method. Average silhouette scores at the moment cannot be used as a cluster validation criterion since they are close to 0.0 indicating that the samples are on or very close to the decision boundary. We would have to perform further analysis with the silhouette coefficient to validate our `elbow` method choice with this criterion, but we will leave this for future work. For now, we will use silhouette scores as complementary information, providing a view on our clusters' performance with this criterion.

We will fit `MiniBatchKMeans` with the decided clusters number:

In [ ]:
clusterer = MiniBatchKMeans(n_clusters=kl.elbow)
# train_clusters = pd.Series(clusterer.fit_predict(norm_train.to_numpy()))
train_cluster_labels = pd.Series(clusterer.fit_predict(train_vals))

train_cluster_labels.shape

(450000,)

In [ ]:
train_cluster_labels.value_counts()

1    64149
4    63555
0    62222
2    61050
6    59945
3    53187
5    50904
7    34988
dtype: int64

In [ ]:
df_clusters = pd.DataFrame({
    "Clusters": train_cluster_labels,
    "TARGET": target,
})

df_clusters.head()

,Clusters,TARGET
0,0,-1
1,6,-1
2,6,-1
3,5,-1
4,1,1


In [ ]:
# Compute population count per cluster for general population (-1) and customers (1)
target_group = df_clusters.groupby(by="TARGET")["Clusters"].value_counts().to_frame(name='Counts').reset_index()
# Calculate percentages per cluster per population
target_group['Perc'] = target_group.groupby("TARGET")['Counts'].apply(lambda x: round(100 * x.astype(float)/x.sum(), 2))

target_group

,TARGET,Clusters,Counts,Perc
0,-1,6,55418,18.47
1,-1,3,51266,17.09
2,-1,4,47796,15.93
3,-1,0,38008,12.67
4,-1,2,31227,10.41
5,-1,7,30985,10.33
6,-1,1,27521,9.17
7,-1,5,17779,5.93
8,1,1,36628,24.42
9,1,5,33125,22.08


In [ ]:
# Plot population per cluster for `AZDIAS` and `CUSTOMERS`
popul_clust_file = f'{colab_dir if IN_COLAB else remote_dir}pop_cluster_bars_{ts}.png'

with sns.axes_style("whitegrid"):
    plt.figure(figsize=(10, 6))
    totals = target_group.groupby("TARGET")['Counts'].sum()
    ax = sns.barplot(x="Clusters", y='Perc', hue="TARGET", data=target_group, palette='Blues_d')
    h, l = ax.get_legend_handles_labels()
    ax.legend(h, ['General', 'Customers'], title="Population")
    ax.set(title='Population portion per cluster', ylabel='Population percentage (%)')
    # add percentage annotations on top of bars
    for tot, p in zip(totals, ax.containers):
        labels = [str(rect.get_height()) + '%' for rect in p]
        for rect, label in zip(p, labels):
            ax.text(rect.get_x() + rect.get_width() / 2,
                    rect.get_height() * 1.01,
                    label,
                    ha="center",
                    va="bottom",
                    fontsize="small",
                    fontweight="semibold"
            )
    # copyright text
    plt.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
    plt.tight_layout()
    plt.savefig(popul_clust_file, transparent=False, bbox_inches="tight")
    plt.close()

In [ ]:
# Create plotting data combining PCA results with K-Means clustering
pcadf = train_reduced.assign(Predicted_cluster=clusterer.labels_, Population=target)
pcadf["Population"] = pcadf["Population"].replace(-1, 'General').replace(1, 'Customers')

In [ ]:
popul_clust_file = f'{colab_dir if IN_COLAB else remote_dir}cluster_results_{ts}.png'

with sns.axes_style("whitegrid"), sns.plotting_context("paper"):
    plt.figure(figsize=(10, 8))
    scatr = sns.scatterplot(x="PC1", y="PC2", data=pcadf, hue="Predicted_cluster", style="Population", palette="Set2", s=50,)
    scatr.set(title='Clustering results from demographics data')
    plt.legend(bbox_to_anchor=(1.01, .6), loc=2, borderaxespad=0.0)
    # copyright text
    plt.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
    plt.tight_layout()
    plt.savefig(popul_clust_file, transparent=False, bbox_inches="tight")
    plt.close()

In [ ]:
# Clusters centroid locations on PCA space
cluster_centroids = pd.DataFrame(clusterer.cluster_centers_, columns=pca_list)
cluster_centroids

,CJT_TYP_2/FINANZ_SPARER,FINANZ_ANLEGER/FINANZ_UNAUFFAELLIGER,OST_WEST_KZ_W/STRUKTURTYP,HH_EINKOMMEN_SCORE/OST_WEST_KZ_O,ANREDE_KZ/SEMIO_KAEM,KBA13_BAUMAX/LP_FAMILIE_FEIN,OST_WEST_KZ_O/KBA13_KW_61_120,PLZ8_ANTG4/KBA13_ANTG4,KBA13_CCM_3001/KBA13_CCM_2501,KBA05_MAXSEG/KBA05_MOTOR,KBA13_ANTG4/PLZ8_ANTG4,KBA05_MAXBJ/SHOPPER_TYP,VERS_TYP/HEALTH_TYP,KBA05_MOD8/KBA05_SEG9,LP_FAMILIE_FEIN/LP_LEBENSPHASE_FEIN,KBA05_KW3/KBA05_CCM4,KBA13_ALTERHALTER_45/KBA13_HALTER_35,KBA13_VORB_1_2/KBA13_ALTERHALTER_61,D19_GESAMT_ANZ_24/D19_VERSAND_ANZ_24,D19_GESAMT_ANZ_24/D19_VERSAND_ANZ_24,KBA13_KRSZUL_NEU/KBA05_MOD8,KBA05_ANHANG/KBA05_ANTG3,KBA05_ANHANG/KBA05_BAUMAX,SHOPPER_TYP/KBA05_ANTG1,SHOPPER_TYP/KBA13_KRSSEG_VAN,KBA05_ANHANG/KBA05_MOTRAD,KBA05_MOTRAD/BALLRAUM,VHN/D19_SOZIALES,PLZ8_ANTG4/KBA05_ANHANG,PLZ8_ANTG4/KBA05_ANTG3,SHOPPER_TYP/KBA13_ANTG4,PLZ8_ANTG4/KBA13_KRSSEG_VAN,D19_LETZTER_KAUF_BRANCHE_D19_UNBEKANNT/D19_KONSUMTYP,KBA05_ANHANG/KBA05_SEG7,KBA05_MOTRAD/KBA05_ANTG3,KBA05_MOTRAD/AGER_TYP,VHN/KBA05_MOTRAD,AGER_TYP/KBA05_ZUL2,KBA05_SEG8/KBA05_SEG7,KBA05_SEG7/AGER_TYP,...,D19_BANKEN_DIREKT/D19_REISEN,D19_REISEN/KBA13_CCM_1500,D19_HAUS_DEKO/D19_BEKLEIDUNG_REST,D19_BEKLEIDUNG_REST/D19_BANKEN_DIREKT,KBA13_KW_70/KBA05_ANTG2,D19_BEKLEIDUNG_REST/D19_HAUS_DEKO,D19_VERSAND_REST/D19_BEKLEIDUNG_REST,KBA05_MOD1/KBA05_SEG10,KK_KUNDENTYP/D19_VERSAND_ANZ_24,KBA05_ANTG2/D19_VOLLSORTIMENT,KBA05_SEG9/D19_TECHNIK,KBA05_ALTER2/D19_VERSAND_ANZ_12,D19_SONSTIGE/D19_VERSAND_ANZ_12,KBA05_SEG9/KBA05_ALTER3,D19_VOLLSORTIMENT/KBA05_HERST5,KBA05_MOD8/D19_REISEN,D19_BUCH_CD/KBA05_KW3,D19_BUCH_CD/KBA05_MOD8,D19_BUCH_CD/KBA05_KRSVAN,D19_VOLLSORTIMENT/D19_GESAMT_ONLINE_QUOTE_12,RELAT_AB/KBA05_KW3,D19_SONSTIGE/EXTSEL992,KBA05_CCM4/D19_SONSTIGE,KBA05_DIESEL/UMFELD_JUNG,KBA05_DIESEL/RT_UEBERGROESSE,KBA13_VORB_2/KBA13_VORB_1_2,GFK_URLAUBERTYP/KBA05_KRSVAN,D19_SONSTIGE/D19_VERSAND_ANZ_24,KBA05_SEG5/KBA13_SEG_OBERKLASSE,KBA05_BAUMAX/UMFELD_ALT,KBA13_KW_90/KBA13_CCM_3001,KBA05_DIESEL/KBA05_HERST4,KBA13_SEG_OBERKLASSE/KBA13_KRSZUL_NEU,KBA05_MOD4/KBA05_SEG5,KBA13_BJ_2008/RETOURTYP_BK_S,KBA05_VORB2/EXTSEL992,KBA05_HERST4/KBA05_FRAU,D19_VERSAND_ONLINE_QUOTE_12/KBA05_KRSVAN,GFK_URLAUBERTYP/D19_VERSAND_ONLINE_QUOTE_12,KK_KUNDENTYP/D19_GESAMT_ANZ_12
0,-1.203329,-0.223992,-0.743978,-0.358931,-0.215974,-0.670338,0.027057,0.045513,-0.050277,-0.068199,0.149190,0.059806,0.010193,0.049111,-0.018365,-0.017170,0.039205,-0.010736,-0.015513,-0.012801,-0.000073,-0.004348,-0.038466,0.002093,-0.035603,0.029497,0.003356,-0.010572,-0.002776,0.007718,0.030059,0.000504,0.040567,-0.015568,-0.025967,0.014422,-0.024002,0.004746,-0.000651,0.002267,...,0.012147,-0.006764,-0.007272,-0.015309,-0.001701,0.000734,-0.015052,-0.017821,-0.004639,-0.000819,-0.010061,-0.008805,0.019471,-0.014947,-0.010884,0.010046,0.002587,-0.004962,0.011280,0.001355,-0.003376,-0.002830,-0.000733,0.022321,-0.007416,-0.015653,0.010489,-0.017771,-0.001920,-0.019528,-0.006500,-0.010605,0.006411,-0.009840,-0.013895,0.006126,-0.005134,0.009664,0.004420,0.002309
1,1.189757,0.597621,-0.734094,-1.377590,0.097498,0.876267,-0.297897,-0.068185,-0.199166,-0.053083,-0.298689,-0.049058,0.019444,-0.050699,-0.014909,-0.057886,0.039705,-0.005297,0.032628,0.018790,0.027643,-0.017474,-0.004252,-0.011265,0.016628,-0.047244,0.017252,0.018758,-0.035465,0.008850,-0.010114,0.009972,-0.002060,0.031978,-0.014015,0.023253,-0.029281,0.003445,0.002080,0.012119,...,-0.006814,0.006297,0.004343,0.001312,-0.016860,0.003112,0.003790,0.000480,0.013166,0.011383,0.007280,-0.010061,-0.009563,-0.001066,0.000470,-0.000179,0.005277,0.000953,0.002640,-0.000785,0.006743,0.008656,-0.000214,-0.008452,0.016615,0.004329,0.005615,-0.004625,0.007773,-0.008486,0.001407,0.011943,-0.003393,0.023931,0.017953,-0.010767,0.018076,0.001721,0.004933,-0.026375
2,-1.543665,0.865122,0.069302,0.639530,0.278657,0.131343,-0.315198,0.126088,-0.006557,-0.094476,-0.007997,-0.027546,0.058509,0.107610,0.066454,0.002132,-0.009539,-0.022984,0.093150,0.070689,0.012023,0.005554,0.018877,-0.038802,-0.0009

In [ ]:
centroid_heatmp_file = f'{colab_dir if IN_COLAB else remote_dir}centroids_heatmap_{ts}.png'

# Plot a heatmap with centroid coordinates of the top 20 most important PCs
with sns.plotting_context("paper"):
    plt.figure(figsize=(8, 6))
    ax = sns.heatmap(cluster_centroids.T.iloc[:20, :], annot=True, cmap='YlGnBu')
    ax.set(xlabel="Cluster", title="Attribute Value by Centroid")
    # copyright text
    plt.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
    plt.tight_layout()
    plt.savefig(centroid_heatmp_file, transparent=False, bbox_inches="tight")
    plt.close()

In [ ]:
# map the cluster labels back to each individual person from the population data
population_transformed = pd.concat([population_transformed, df_clusters], axis=1)
# Add the LNR attribute to be able to associate with original data
population_transformed = population_transformed.assign(LNR=clientIDs)
population_transformed.head()

,CJT_TYP_2/FINANZ_SPARER,FINANZ_ANLEGER/FINANZ_UNAUFFAELLIGER,OST_WEST_KZ_W/STRUKTURTYP,HH_EINKOMMEN_SCORE/OST_WEST_KZ_O,ANREDE_KZ/SEMIO_KAEM,KBA13_BAUMAX/LP_FAMILIE_FEIN,OST_WEST_KZ_O/KBA13_KW_61_120,PLZ8_ANTG4/KBA13_ANTG4,KBA13_CCM_3001/KBA13_CCM_2501,KBA05_MAXSEG/KBA05_MOTOR,KBA13_ANTG4/PLZ8_ANTG4,KBA05_MAXBJ/SHOPPER_TYP,VERS_TYP/HEALTH_TYP,KBA05_MOD8/KBA05_SEG9,LP_FAMILIE_FEIN/LP_LEBENSPHASE_FEIN,KBA05_KW3/KBA05_CCM4,KBA13_ALTERHALTER_45/KBA13_HALTER_35,KBA13_VORB_1_2/KBA13_ALTERHALTER_61,D19_GESAMT_ANZ_24/D19_VERSAND_ANZ_24,D19_GESAMT_ANZ_24/D19_VERSAND_ANZ_24,KBA13_KRSZUL_NEU/KBA05_MOD8,KBA05_ANHANG/KBA05_ANTG3,KBA05_ANHANG/KBA05_BAUMAX,SHOPPER_TYP/KBA05_ANTG1,SHOPPER_TYP/KBA13_KRSSEG_VAN,KBA05_ANHANG/KBA05_MOTRAD,KBA05_MOTRAD/BALLRAUM,VHN/D19_SOZIALES,PLZ8_ANTG4/KBA05_ANHANG,PLZ8_ANTG4/KBA05_ANTG3,SHOPPER_TYP/KBA13_ANTG4,PLZ8_ANTG4/KBA13_KRSSEG_VAN,D19_LETZTER_KAUF_BRANCHE_D19_UNBEKANNT/D19_KONSUMTYP,KBA05_ANHANG/KBA05_SEG7,KBA05_MOTRAD/KBA05_ANTG3,KBA05_MOTRAD/AGER_TYP,VHN/KBA05_MOTRAD,AGER_TYP/KBA05_ZUL2,KBA05_SEG8/KBA05_SEG7,KBA05_SEG7/AGER_TYP,...,D19_BEKLEIDUNG_REST/D19_BANKEN_DIREKT,KBA13_KW_70/KBA05_ANTG2,D19_BEKLEIDUNG_REST/D19_HAUS_DEKO,D19_VERSAND_REST/D19_BEKLEIDUNG_REST,KBA05_MOD1/KBA05_SEG10,KK_KUNDENTYP/D19_VERSAND_ANZ_24,KBA05_ANTG2/D19_VOLLSORTIMENT,KBA05_SEG9/D19_TECHNIK,KBA05_ALTER2/D19_VERSAND_ANZ_12,D19_SONSTIGE/D19_VERSAND_ANZ_12,KBA05_SEG9/KBA05_ALTER3,D19_VOLLSORTIMENT/KBA05_HERST5,KBA05_MOD8/D19_REISEN,D19_BUCH_CD/KBA05_KW3,D19_BUCH_CD/KBA05_MOD8,D19_BUCH_CD/KBA05_KRSVAN,D19_VOLLSORTIMENT/D19_GESAMT_ONLINE_QUOTE_12,RELAT_AB/KBA05_KW3,D19_SONSTIGE/EXTSEL992,KBA05_CCM4/D19_SONSTIGE,KBA05_DIESEL/UMFELD_JUNG,KBA05_DIESEL/RT_UEBERGROESSE,KBA13_VORB_2/KBA13_VORB_1_2,GFK_URLAUBERTYP/KBA05_KRSVAN,D19_SONSTIGE/D19_VERSAND_ANZ_24,KBA05_SEG5/KBA13_SEG_OBERKLASSE,KBA05_BAUMAX/UMFELD_ALT,KBA13_KW_90/KBA13_CCM_3001,KBA05_DIESEL/KBA05_HERST4,KBA13_SEG_OBERKLASSE/KBA13_KRSZUL_NEU,KBA05_MOD4/KBA05_SEG5,KBA13_BJ_2008/RETOURTYP_BK_S,KBA05_VORB2/EXTSEL992,KBA05_HERST4/KBA05_FRAU,D19_VERSAND_ONLINE_QUOTE_12/KBA05_KRSVAN,GFK_URLAUBERTYP/D19_VERSAND_ONLINE_QUOTE_12,KK_KUNDENTYP/D19_GESAMT_ANZ_12,Clusters,TARGET,LNR
0,-1.234629,-0.867582,-0.578983,-0.220230,-0.742176,0.211376,-0.222328,0.117435,0.625594,-0.276393,0.209671,1.080525,0.073092,-0.005344,-0.725521,0.534920,-0.816195,0.000018,-0.548220,-0.422507,0.358123,0.910408,-0.577757,-0.334013,-0.317721,-0.865876,-0.034087,0.055927,0.069684,0.000181,0.192486,0.326819,-0.207724,0.509368,-0.385418,0.339499,0.397615,0.350358,-0.024274,0.040678,...,0.491691,0.887629,0.179989,0.231284,0.223842,0.026079,-0.465061,-0.227642,-0.249795,-0.011948,0.248586,-0.388448,0.285329,-0.139759,-0.337783,-0.083996,-0.233873,-0.179353,-0.448052,-0.513697,-0.347848,-0.781476,0.001404,-0.283709,-0.331719,0.221276,-0.163244,0.291595,0.186463,-0.005497,-0.228021,0.307563,-0.713716,0.401752,0.477682,-0.182475,-0.062534,0,-1,780657
1,0.977389,0.134613,0.048990,-0.160303,-0.804427,-0.352177,0.670749,-1.172047,-1.055501,0.573792,0.661885,-1.608553,0.358878,0.913354,-1.173424,0.805668,-0.148719,0.882287,0.335386,0.451623,0.064425,1.197852,-0.666765,0.302235,0.656651,0.195785,-0.440172,0.129148,-0.404084,0.013804,-0.379258,0.256196,-0.816625,0.160029,0.401891,-0.057911,0.288800,0.054266,0.203565,-0.225481,...,0.597577,0.099311,0.298713,0.140128,0.083948,-0.230444,0.391376,0.510545,0.052131,0.096806,-0.743018,0.162313,-0.162888,-0.008984,0.063964,-0.152078,-0.096144,-0.237473,0.091904,0.133245,0.210970,-0.439297,0.643481,0.427124,0.061224,0.371090,-0.009132,-0.335295,-0.327904,0.320068,-0.003117,-0.307557,-0.310712,-0.153566,0.351870,-0.000796,0.130907,6,-1,283232
2,0.640617,0.397748,0.081203,-0.250016,0.403747,-1.531851,-0.125694,0.206615,1.062772,-0.607811,-0.043918,-0.782834,0.357914,0.292886,-0.547790,-0.100105,-0.214742,0.114630,-0.347420,-0.215213,0.445487,-0.195424,-0.093167,-0.405749,0.601990,-0.136346,-0.291381,0.456311,-0.518213,-0.715454,-0.119906,-0.011904,-0.492868,-0.152883,-0.046090,0.26

Now we can examine which individuals are grouped together in the same cluster by looking at one of the clusters in more detail. If we take cluster 2 for example and look at the location of the centroid in the heatmap above, it tells us that it has the highest value for the attributes:

In [ ]:
cluster_centroids.T[1].nlargest(1).index[0]

'CJT_TYP_2/FINANZ_SPARER'

In addition, we can check the individuals that fit that description:

In [ ]:
cluster_2 = population_transformed[population_transformed['Clusters'] == 2]
print("Individuals in cluster: ", cluster_2.shape[0])
print("\nIndividuals `Customers`: ", cluster_2[cluster_2['TARGET'] == 1].shape[0])
print("\nIndividuals `General`: ", cluster_2[cluster_2['TARGET'] == -1].shape[0])
print(30*'-' + '\n')
cluster_2.head()

Individuals in cluster:  61050

Individuals `Customers`:  29823

Individuals `General`:  31227
------------------------------



,CJT_TYP_2/FINANZ_SPARER,FINANZ_ANLEGER/FINANZ_UNAUFFAELLIGER,OST_WEST_KZ_W/STRUKTURTYP,HH_EINKOMMEN_SCORE/OST_WEST_KZ_O,ANREDE_KZ/SEMIO_KAEM,KBA13_BAUMAX/LP_FAMILIE_FEIN,OST_WEST_KZ_O/KBA13_KW_61_120,PLZ8_ANTG4/KBA13_ANTG4,KBA13_CCM_3001/KBA13_CCM_2501,KBA05_MAXSEG/KBA05_MOTOR,KBA13_ANTG4/PLZ8_ANTG4,KBA05_MAXBJ/SHOPPER_TYP,VERS_TYP/HEALTH_TYP,KBA05_MOD8/KBA05_SEG9,LP_FAMILIE_FEIN/LP_LEBENSPHASE_FEIN,KBA05_KW3/KBA05_CCM4,KBA13_ALTERHALTER_45/KBA13_HALTER_35,KBA13_VORB_1_2/KBA13_ALTERHALTER_61,D19_GESAMT_ANZ_24/D19_VERSAND_ANZ_24,D19_GESAMT_ANZ_24/D19_VERSAND_ANZ_24,KBA13_KRSZUL_NEU/KBA05_MOD8,KBA05_ANHANG/KBA05_ANTG3,KBA05_ANHANG/KBA05_BAUMAX,SHOPPER_TYP/KBA05_ANTG1,SHOPPER_TYP/KBA13_KRSSEG_VAN,KBA05_ANHANG/KBA05_MOTRAD,KBA05_MOTRAD/BALLRAUM,VHN/D19_SOZIALES,PLZ8_ANTG4/KBA05_ANHANG,PLZ8_ANTG4/KBA05_ANTG3,SHOPPER_TYP/KBA13_ANTG4,PLZ8_ANTG4/KBA13_KRSSEG_VAN,D19_LETZTER_KAUF_BRANCHE_D19_UNBEKANNT/D19_KONSUMTYP,KBA05_ANHANG/KBA05_SEG7,KBA05_MOTRAD/KBA05_ANTG3,KBA05_MOTRAD/AGER_TYP,VHN/KBA05_MOTRAD,AGER_TYP/KBA05_ZUL2,KBA05_SEG8/KBA05_SEG7,KBA05_SEG7/AGER_TYP,...,D19_BEKLEIDUNG_REST/D19_BANKEN_DIREKT,KBA13_KW_70/KBA05_ANTG2,D19_BEKLEIDUNG_REST/D19_HAUS_DEKO,D19_VERSAND_REST/D19_BEKLEIDUNG_REST,KBA05_MOD1/KBA05_SEG10,KK_KUNDENTYP/D19_VERSAND_ANZ_24,KBA05_ANTG2/D19_VOLLSORTIMENT,KBA05_SEG9/D19_TECHNIK,KBA05_ALTER2/D19_VERSAND_ANZ_12,D19_SONSTIGE/D19_VERSAND_ANZ_12,KBA05_SEG9/KBA05_ALTER3,D19_VOLLSORTIMENT/KBA05_HERST5,KBA05_MOD8/D19_REISEN,D19_BUCH_CD/KBA05_KW3,D19_BUCH_CD/KBA05_MOD8,D19_BUCH_CD/KBA05_KRSVAN,D19_VOLLSORTIMENT/D19_GESAMT_ONLINE_QUOTE_12,RELAT_AB/KBA05_KW3,D19_SONSTIGE/EXTSEL992,KBA05_CCM4/D19_SONSTIGE,KBA05_DIESEL/UMFELD_JUNG,KBA05_DIESEL/RT_UEBERGROESSE,KBA13_VORB_2/KBA13_VORB_1_2,GFK_URLAUBERTYP/KBA05_KRSVAN,D19_SONSTIGE/D19_VERSAND_ANZ_24,KBA05_SEG5/KBA13_SEG_OBERKLASSE,KBA05_BAUMAX/UMFELD_ALT,KBA13_KW_90/KBA13_CCM_3001,KBA05_DIESEL/KBA05_HERST4,KBA13_SEG_OBERKLASSE/KBA13_KRSZUL_NEU,KBA05_MOD4/KBA05_SEG5,KBA13_BJ_2008/RETOURTYP_BK_S,KBA05_VORB2/EXTSEL992,KBA05_HERST4/KBA05_FRAU,D19_VERSAND_ONLINE_QUOTE_12/KBA05_KRSVAN,GFK_URLAUBERTYP/D19_VERSAND_ONLINE_QUOTE_12,KK_KUNDENTYP/D19_GESAMT_ANZ_12,Clusters,TARGET,LNR
10,-2.640672,0.843719,0.102702,-0.075584,-0.362793,0.264781,-0.223328,-0.506855,0.951140,-0.468598,0.486646,0.660295,-0.003846,0.296077,-0.272760,0.113319,0.108795,0.099468,0.122817,0.083201,1.247350,-0.455726,-0.571825,-0.067285,0.560072,-0.089024,0.193925,-0.005272,0.283273,0.301201,-0.373616,0.631035,-0.102140,-0.161052,0.052280,0.024935,-0.390521,-0.242384,-0.069935,-0.120373,...,-0.093179,-0.266858,0.061900,-0.121609,-0.099128,1.126712,0.046679,-0.203011,-0.181079,0.299730,-0.202914,-0.123354,-0.024537,-0.002241,-0.569587,0.333229,0.299639,0.236294,0.160245,0.115646,0.066705,-0.318140,-0.009263,-0.165102,-0.293401,0.003036,0.051617,-0.015282,0.585075,0.076598,-0.012210,-0.207770,0.055532,0.163585,0.575585,-0.187314,-0.558813,2,1,24700
11,-2.063223,0.400269,0.857781,0.483734,-0.047520,0.626514,-0.435673,0.023308,0.416930,-0.389410,-0.945167,-0.911041,0.583819,1.262275,-0.467349,0.022939,-0.079326,0.396674,0.914962,0.051674,0.387914,0.044065,-0.259569,-0.227781,-0.473022,0.260998,-0.377862,0.094678,0.330117,-0.230632,-0.042768,-0.003732,-0.358954,0.082959,0.452334,-0.659340,-0.557442,0.425277,-0.677737,0.100660,...,0.190761,0.065015,0.571585,0.325712,-0.128486,-0.151182,0.555767,-0.171523,-0.063445,-0.067942,-0.042285,0.232594,-0.314477,-0.153548,0.298703,0.239172,-0.167129,-0.444540,0.722695,0.113475,0.369564,-0.260474,-0.217681,0.462359,-0.165925,-0.070592,-0.045608,0.159999,-0.279164,-0.239771,-0.236415,0.097393,0.292148,-0.015301,-0.459012,0.149128,-0.046471,2,1,164256
18,-1.372159,1.166249,-1.043671,0.247380,0.794591,-0.903142,-1.727027,1.351772,-0.244045,0.745800,0.651984,-0.782614,0.237077,-0.593261,-0.004930,-0.168611,0.727278,-0.001004,0.176355,-0.105044,-0.082041,1.114180,0.089635,-0.294440,-0.417220,0.283097,-0.283342,0.296308,0.666146,0.279036,-0.544161,-0.391942,-0.175988,-0.031520,0.484813,-

We have created a list of the two most influencing attributes characterising each principal component. We will create a dictionary mapping of these two attributes for each PC, extracting more available information from the excel files provided for the attributes' explanation. This will give us more information on the dominant context of each PC.

In [ ]:
# For each name we created for PCs we check if the attribute exists in the excel file and extract descriptions
pca_list_map = {pc: ', '.join([dfdias_attrib[dfdias_attrib["Attribute"].isin([i, i+'_RZ'])]["Description"].iloc[0] \
                               if dfdias_attrib["Attribute"].isin([i, i+'_RZ']).any() else 'UNKNOWN' for i in pc.split('/')]) \
                for pc in pca_list}

list(pca_list_map.items())[:3]

[('CJT_TYP_2/FINANZ_SPARER', 'UNKNOWN, financial typology: money saver'),
 ('FINANZ_ANLEGER/FINANZ_UNAUFFAELLIGER',
  'financial typology: investor, financial typology: unremarkable'),
 ('OST_WEST_KZ_W/STRUKTURTYP', 'UNKNOWN, UNKNOWN')]

The above mapping allows us to further look into the characteristics of a cluster. For example for cluster 2 we can look for the location of the highest (most dominant) centroid coordinates in the heatmap above. This tells us that it has the highest value in some principal components. We can extract these names and look at the dictionary we created for more info.

In [ ]:
# Cluster 2
cluster2_top5_pc = cluster_centroids.T[1].sort_values(ascending=False).head().index.to_list()

for pc in cluster2_top5_pc:
    print("[Attributes]- {},\t[Description]- {}".format(pc, pca_list_map[pc]))

[Attributes]- CJT_TYP_2/FINANZ_SPARER,	[Descritpion]- UNKNOWN, financial typology: money saver
[Attributes]- KBA13_BAUMAX/LP_FAMILIE_FEIN,	[Descritpion]- UNKNOWN, familytyp fine
[Attributes]- FINANZ_ANLEGER/FINANZ_UNAUFFAELLIGER,	[Descritpion]- financial typology: investor, financial typology: unremarkable
[Attributes]- ANREDE_KZ/SEMIO_KAEM,	[Descritpion]- gender, affinity indicating in what way the person is of a fightfull attitude
[Attributes]- KBA05_BAUMAX/KBA05_ANTG3,	[Descritpion]- most common building-type within the cell, number of 6-10 family houses in the cell


From the above, on first look we can extract characteristics of **Cluster 2**  group such as:
> *Individuals that are money savers and investors, are in a good financial status, living in the suburbs, or sparse areas*.

If we further investigate the unknown attributes we find:

* `CJT_TYP_2`: relating to the preferred information and buying channels of consumers
* `KBA13_BAUMAX`: is the estimated share of >10 family houses in the PLZ8.

With this we can add to the above cluster description:

> *Individuals that are money savers and investors, are in a good financial status, living in the suburbs, or sparse areas …in common houses with buying preference channel 2 (Advertising- and Consumptiontraditionalist)*.

Similarly for **Cluster 1**:


In [ ]:
# Cluster 1
cluster1_top5_pc = cluster_centroids.T[0].sort_values(ascending=False).head().index.to_list()

for pc in cluster1_top5_pc:
    print("[Attributes]- {},\t[Description]- {}".format(pc, pca_list_map[pc]))

[Attributes]- KBA13_ANTG4/PLZ8_ANTG4,	[Descritpion]- UNKNOWN, number of >10 family houses in the PLZ8
[Attributes]- KBA05_MAXBJ/SHOPPER_TYP,	[Descritpion]- most common age of the cars in the microcell, shopping typology
[Attributes]- KBA05_MOD8/KBA05_SEG9,	[Descritpion]- share of vans (in an AZ specific definition), share of vans in the microcell
[Attributes]- PLZ8_ANTG4/KBA13_ANTG4,	[Descritpion]- number of >10 family houses in the PLZ8, UNKNOWN
[Attributes]- GEBAEUDETYP/VHN,	[Descritpion]- type of building (residential or commercial), UNKNOWN


> *Individuals that live in not very dense areas, in residential or commercial buildings, with most common cars and vans*.

We may also drill down to specifics of each attribute:

In [ ]:
train = train.assign(LNR=clientIDs, TARGET=target)

In [ ]:
# E.g. FINANZ_ANLEGER
# Check the population in Cluster 1 for PC: CJT_TYP_2/FINANZ_ANLEGER
cluster_1 = population_transformed[population_transformed['Clusters'] == 1]['LNR']
# Then filter the original data by these ids and get percentages:
train[train['LNR'].isin(cluster_1)]['FINANZ_ANLEGER'].value_counts(sort=False, normalize=True).mul(100).round(1)

1     0.0
2     0.2
3     0.4
4     0.9
5    98.5
Name: FINANZ_ANLEGER, dtype: float64

So, in this cluster we have these kinds of `investor` in general. We could also examine subsets based on general population and customers

In [ ]:
group_cl1 = train[train['LNR'].isin(cluster_1)].groupby('TARGET')
# General population dataset
group_cl1.get_group((list(group_cl1.groups)[0]))['FINANZ_ANLEGER'].value_counts(sort=False, normalize=True).mul(100).round(1)

1     0.1
2     0.3
3     0.9
4     2.0
5    96.7
Name: FINANZ_ANLEGER, dtype: float64

For the **general population** the percentages of `investor` in cluster 1 are as above.

In [ ]:
# Customers dataset
group_cl1.get_group((list(group_cl1.groups)[1]))['FINANZ_ANLEGER'].value_counts(sort=False, normalize=True).mul(100).round(1)

1     0.0
2     0.0
3     0.1
4     0.1
5    99.8
Name: FINANZ_ANLEGER, dtype: float64

For the **customers population** the percentages of `investor` in cluster 1 are as above.

### Segmentation conclusion

The parts of the general population that may be considered more likely to be part of the mail-order company's main customer base are the ones in clusters:

In [ ]:
top_customer_clusters = target_group.loc[(target_group["TARGET"] == 1), ["Clusters", "Perc"]].nlargest(2, "Perc").reset_index(drop=True)
top_customer_clusters

,Clusters,Perc
0,1,24.42
1,5,22.08


These are the clusters with the highest customers population concentration. We may consider that these clusters in the corresponding general population will provide the highest chances for acquiring new customers. If we like to have a greater level of detail, we could transform the data to fit in more clusters. We could then target more specific segments of the general population.

In [ ]:
print("Elapsed time: ", str(timedelta(seconds=(timer() - start_run))))

Elapsed time:  0:06:39.358728


In [ ]:
del train, population_transformed, cluster_2, cluster_1, cluster_centroids, pcadf, pca_df, train_vals, \
clusterer, all_Kscores,plt_attrs_wgts, train_reduced, pca_wghts_df, norm_train, pca, azdias, custrs
gc.collect()

28066

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

###Feature selection using [**LightGBM**](https://lightgbm.readthedocs.io/en/latest/index.html), a gradient boosting framework that uses tree-based learning algorithms.

We will use **feature importances** of a LightGBM model with a threshold value to remove attributes not considered important by the model. We will achieve this by removing all zero importance features (as computed by LightGBM) from the model.

The evaluation metric we will use is ROC_AUC_SCORE:

An ROC curve (receiver operating characteristic curve) is a graph showing the performance of a classification model at all classification thresholds. This curve plots two parameters:

* True Positive Rate
* False Positive Rate

AUC measures the entire two-dimensional area underneath the entire ROC curve  from (0,0) to (1,1). AUC provides an aggregate measure of performance across all possible classification thresholds.

* AUC is scale-invariant. It measures how well predictions are ranked, rather than their absolute values.
* AUC is classification-threshold-invariant. It measures the quality of the model's predictions irrespective of what classification threshold is chosen.

We will use [LGBMClassifier](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html), a Gradient Boosted Model from the LightGBM library to assess feature importances.

LGB plotting and interpretation:
* https://stackoverflow.com/questions/61862597/how-to-get-variance-explained-of-features-for-lightgbm-best_iter
* https://stackoverflow.com/questions/53413701/feature-importance-using-lightgbm

KFOLDS

* https://scikit-learn.org/stable/modules/cross_validation.html#k-fold

In [ ]:
# Add the `LNR` column.
df_train = df_train.assign(LNR=clientIDs.values)
df_train.to_csv(f'{colab_dir if IN_COLAB else remote_dir}df_train_{ts}.csv', index=False)

In [ ]:
# df_train = pd.read_csv(f'{colab_dir if IN_COLAB else remote_dir}' + 'df_train_<add ts>.csv')
# df_train.shape

In [ ]:
# Create train and target sets.
train = df_train.drop(columns=["TARGET"])
target = df_train["TARGET"]
target[target == -1] = 0

# Add the `LNR` column.
train = train.assign(LNR=clientIDs.values)

train.shape

(450000, 377)

So, from feature selection using gradient boosting model we may keep:

* the list that contains all attributes identified with zero importance.
* a list that contains a smaller subset of non zero importance attributes reaching 0.95 cumulative importance.

We may test the above to other sample subsets. If it turns out that the feature selection works well we can also use the model above for making predictions in later steps.

For this we will create a couple more functions to help us through the steps.

In [ ]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, auc, RocCurveDisplay
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def rf_model(features_labels_split, rf_params, n_folds=3):
    """Train and test a light gradient boosting model using
    cross validation with KFold splits. 
    
    Args:
        features_labels_split (list): List containing train-test split of inputs.
            Either [X_train, X_test, y_train, y_test] or [features, test_features]
            features will be pd.DataFrame of training features to use for training
            a model. Must include the `RESPONSE` column.
            test_features will be pd.DataFrame of testing features to use for
            making predictions with the model.
        lgbm_params (dict): LightGBM classifier init parameters.
        n_folds (int, default = 5): number of folds to use for cross validation.
        
    Returns:
        submission (pd.DataFrame): dataframe with `LNR` and `RESPONSE`
            probabilities predicted by the model.
        attrib_imp (pd.DataFrame): dataframe with the feature
            importances from the model.
        valid_metrics (pd.DataFrame): dataframe with training and validation
            metrics (ROC AUC) for each fold and overall.
    
    Ref: https://www.kaggle.com/willkoehrsen/introduction-to-feature-selection#Remove-Collinear-Variables
    """
    rf_ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    # Differentiate assignments according to input (length=4 -> train_test_split)
    if len(features_labels_split) == 4:
        features, test_features, labels, test_labels = features_labels_split
    else:
        features, test_features = features_labels_split
        # Extract the labels for training
        labels = features['RESPONSE']
        test_labels = None

    # Extract the client IDs
    train_ids = features['LNR']
    test_ids = test_features['LNR']
    # Drop the client IDs and datetimes
    features.drop(columns=['LNR', 'EINGEFUEGT_AM', 'RESPONSE'], errors='ignore', inplace=True)
    test_features.drop(columns=['LNR', 'EINGEFUEGT_AM'], errors='ignore', inplace=True)
        
    print('\nTraining Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = features.columns.tolist()
    
    # Impute missing values with Random Selection
    traintestX = [i.apply(imput_nan_rand) for i in (features, test_features)]
    # Normalization
    features, test_features = [scaler.fit_transform(i) for i in traintestX]

    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Create the kfold object
    print("Creating KFold object")
    folds = KFold(n_splits=n_folds)
    
    # Lists for recording validation and training scores
    train_scores = []
    test_scores = []
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    # save figure location
    roc_plot_file = f'{colab_dir if IN_COLAB else remote_dir}rf_roc_{ts}_{rf_ts}.png'
    print("\nStarting fold iterations")
    print(25*'-')
    fig, ax = plt.subplots(figsize=(8,5))
    # Iterate through folds
    for i, (train_idx, val_idx) in enumerate(folds.split(features)):
        # Training data for the fold
        fold_train_X, fold_train_y = features[train_idx], labels.iloc[train_idx]
        # Validation data for the fold
        fold_test_X, fold_test_y = features[val_idx], labels.iloc[val_idx]
        # Create the model
        rf_clf = RandomForestClassifier(**rf_params)
        # Train the model
        rf_clf.fit(fold_train_X, fold_train_y)
        # Extract and keep feature importances
        feature_importance_values += rf_clf.feature_importances_ / folds.n_splits
        # Make predictions using these importances
        test_predictions += rf_clf.predict_proba(test_features)[:, 1] / folds.n_splits
        # Record the out of fold predictions
        y_pred = rf_clf.predict(fold_test_X)
        y_proba = rf_clf.predict_proba(fold_test_X)
        out_of_fold[val_idx] = y_proba[:, 1]
        # Record train and ROC AUC scores
        train_scores.append(roc_auc_score(fold_train_y, rf_clf.predict_proba(fold_train_X)[:, 1]))
        test_scores.append(roc_auc_score(fold_test_y, y_proba[:, 1]))
        viz = RocCurveDisplay.from_predictions(
            fold_test_y,
            y_proba[:, 1],
            name="ROC fold {}".format(i),
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)
        # Clean up memory
        del rf_clf, fold_train_X, fold_test_X, fold_train_y, fold_test_y
        gc.collect()
    
    ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )

    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        title="RandomForest - Receiver operating characteristic (ROC)",
    )
    ax.legend(loc="lower right")
    # copyright text
    plt.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
    plt.tight_layout()
    plt.savefig(roc_plot_file, transparent=False, bbox_inches="tight")
    plt.close()

    print(25*'-')
    print("Creating output")
    # Make the submission dataframe
    submission = pd.DataFrame({'LNR': test_ids, 'RESPONSE': test_predictions})

    # Make the feature importance dataframe
    attrib_imp = pd.DataFrame({'attribute': feature_names,
                               'importance': feature_importance_values})
    
    # Overall validation score
    test_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    test_scores.append(test_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'test': test_scores})
    
    return submission, attrib_imp, metrics

In [ ]:
rf_params = {
    'class_weight': 'balanced',
}

# submit_i, attrib_imp_i, metrics_i = rf_model(train_test_split(train, target, stratify=target, test_size=.25), rf_params)


Training Data Shape:  (262500, 376)
Testing Data Shape:  (87500, 376)
Creating KFold object

Starting fold iterations
-------------------------
-------------------------
Creating output


In [ ]:
# LightGBM
import lightgbm as lgb

def lgb_model(features_labels_split, lgbm_params, n_folds=5):
    """Train and test a light gradient boosting model using
    cross validation with KFold splits. 
    
    Args:
        features_labels_split (list): List containing train-test split of inputs.
            Either [X_train, X_test, y_train, y_test] or [features, test_features]
            features will be pd.DataFrame of training features to use for training
            a model. Must include the `RESPONSE` column.
            test_features will be pd.DataFrame of testing features to use for
            making predictions with the model.
        lgbm_params (dict): LightGBM classifier init parameters.
        n_folds (int, default = 5): number of folds to use for cross validation.
        
    Returns:
        submission (pd.DataFrame): dataframe with `LNR` and `RESPONSE`
            probabilities predicted by the model.
        attrib_imp (pd.DataFrame): dataframe with the feature
            importances from the model.
        valid_metrics (pd.DataFrame): dataframe with training and validation
            metrics (ROC AUC) for each fold and overall.
    
    Ref: https://www.kaggle.com/willkoehrsen/introduction-to-feature-selection#Remove-Collinear-Variables
    """
    lgbm_ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    # Differentiate assignments according to input (length=4 -> train_test_split)
    if len(features_labels_split) == 4:
        features, test_features, labels, test_labels = features_labels_split
    else:
        features, test_features = features_labels_split
        # Extract the labels for training
        labels = features['RESPONSE']

    # Extract the client IDs
    train_ids = features['LNR']
    test_ids = test_features['LNR']
    # Drop the client IDs and datetimes
    features.drop(columns=['LNR', 'EINGEFUEGT_AM', 'RESPONSE'], errors='ignore', inplace=True)
    test_features.drop(columns=['LNR', 'EINGEFUEGT_AM'], errors='ignore', inplace=True)
        
    print('\nTraining Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = features.columns.tolist()
    
    # Impute missing values with Random Selection
    traintestX = [i.apply(imput_nan_rand) for i in (features, test_features)]
    # Normalization
    features, test_features = [scaler.fit_transform(i) for i in traintestX]

    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Create the kfold object
    print("Creating KFold object")
    folds = KFold(n_splits=n_folds)
    
    # Lists for recording validation and training scores
    train_scores = []
    test_scores = []
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    # save figure location
    lgb_roc_plt_file = f'{colab_dir if IN_COLAB else remote_dir}lgb_roc_{ts}_{lgbm_ts}.png'
    print("\nStarting fold iterations")
    print(25*'-')
    fig, ax = plt.subplots(figsize=(8,5))
    # Iterate through folds
    for i, (train_idx, val_idx) in enumerate(folds.split(features)):
        # Training data for the fold
        fold_train_X, fold_train_y = features[train_idx], labels.iloc[train_idx]
        # Validation data for the fold
        fold_test_X, fold_test_y = features[val_idx], labels.iloc[val_idx]
        # Create the model
        lgbm_clf = lgb.LGBMClassifier(**lgbm_params)
        # Train the model
        lgbm_clf.fit(
            fold_train_X, fold_train_y,
            eval_metric='auc',
            eval_set=[(fold_test_X, fold_test_y), (fold_train_X, fold_train_y)],
            eval_names=['test', 'train'],
            early_stopping_rounds=100,
            verbose = 200
        )
        # Keep the best iteration
        best_iter = lgbm_clf.best_iteration_
        # Extract and keep feature importances
        feature_importance_values += lgbm_clf.feature_importances_ / folds.n_splits
        # Make predictions using these importances
        test_predictions += lgbm_clf.predict_proba(test_features, num_iteration=best_iter)[:, 1] / folds.n_splits
        # Record the out of fold predictions
        y_proba = lgbm_clf.predict_proba(fold_test_X, num_iteration=best_iter)
        out_of_fold[val_idx] = y_proba[:, 1]
        # Record the best score
        train_scores.append(lgbm_clf.best_score_['train']['auc'])
        test_scores.append(lgbm_clf.best_score_['test']['auc'])
        viz = RocCurveDisplay.from_predictions(
            fold_test_y,
            y_proba[:, 1],
            name="ROC fold {}".format(i),
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)
        # Clean up memory
        del lgbm_clf, fold_train_X, fold_test_X, fold_train_y, fold_test_y
        gc.collect()

    ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )

    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        title="LightGBM - Receiver operating characteristic (ROC)",
    )
    ax.legend(loc="lower right")
    # copyright text
    plt.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
    plt.tight_layout()
    plt.savefig(lgb_roc_plt_file, transparent=False, bbox_inches="tight")
    plt.close()

    print(25*'-')
    print("Creating output")    
    # Make the submission dataframe
    submission = pd.DataFrame({'LNR': test_ids, 'RESPONSE': test_predictions})
    
    # Make the feature importance dataframe
    attrib_imp = pd.DataFrame({'attribute': feature_names,
                               'importance': feature_importance_values})
    
    # Overall validation score
    test_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    test_scores.append(test_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'test': test_scores}) 
    
    return submission, attrib_imp, metrics

"Ensembles like LGBM build trees in iterations, and each new tree is used to correct the “errors” of the previous trees. This approach is fast and powerful, and prone to overfitting.
That’s why gradient boosted ensembles have a learning_rate parameter that controls the learning speed. Typical values lie within 0.01 and 0.3, but it is possible to go beyond these, especially towards 0.
So, the perfect setup for these 2 parameters (n_estimators and learning_rate) is to use many trees with early stopping and set a low value for learning_rate."

https://towardsdatascience.com/kagglers-guide-to-lightgbm-hyperparameter-tuning-with-optuna-in-2021-ed048d9838b5

In [ ]:
lgbm_params = {
    'boosting_type': 'goss',
    'learning_rate': 0.05,
    'n_estimators': 10000,
    'objective': 'binary',
    'class_weight': 'balanced',
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': 67,
    'n_jobs': -1,
}

srch_params = {
    'boosting_type': 'gbdt',
    'num_leaves': 100,
    'max_depth': 13,
    'learning_rate': 0.3,
    'n_estimators': 10000,
    'objective': 'binary',
    'is_unbalance': True,
    'subsample': .8,
    'subsample_freq': 1,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': 67,
    'n_jobs': -1,
}

submit_i, attrib_imp_i, metrics_i = lgb_model(train_test_split(train, target, stratify=target, test_size=.25), lgbm_params)


Training Data Shape:  (337500, 376)
Testing Data Shape:  (112500, 376)
Creating KFold object

Starting fold iterations
-------------------------
Training until validation scores don't improve for 100 rounds.
[200]	train's auc: 0.929145	train's binary_logloss: 0.339655	test's auc: 0.922348	test's binary_logloss: 0.354317
[400]	train's auc: 0.936108	train's binary_logloss: 0.324076	test's auc: 0.923682	test's binary_logloss: 0.350557
Early stopping, best iteration is:
[480]	train's auc: 0.938517	train's binary_logloss: 0.318646	test's auc: 0.923901	test's binary_logloss: 0.349637
Training until validation scores don't improve for 100 rounds.
[200]	train's auc: 0.928541	train's binary_logloss: 0.341116	test's auc: 0.925391	test's binary_logloss: 0.348965
[400]	train's auc: 0.935509	train's binary_logloss: 0.325617	test's auc: 0.926226	test's binary_logloss: 0.34578
Early stopping, best iteration is:
[487]	train's auc: 0.937958	train's binary_logloss: 0.320037	test's auc: 0.926275	test's 

In [ ]:
# metrics_i.to_csv(f'{colab_dir if IN_COLAB else remote_dir}train_metricsLGBM_{ts}.csv', index=False)
metrics_i

,fold,train,test
0,0,0.938517,0.923901
1,1,0.937958,0.926275
2,2,0.938797,0.924384
3,3,0.938604,0.926604
4,4,0.942619,0.924531
5,overall,0.939299,0.925134


Our model has identified **zero importance** attributes that we may use to further reduce the number of attributes in our datasets.

In [ ]:
def plot_attr_imp(df, n_attrs=15, thres=.95, fig_size=(10, 5), save_file=None):
    """Plots n_attrs most important features and the cumulative importance of
    attributes.Prints the number of features needed to reach the threshold of
    cumulative importance.
    
    Args:
        df (pd.DataFrame): A dataframe of attribute importances.
            Columns must be `attribute` and `importance`.
        n_attrs (int): Number of attributes to plot.
        thres (float): Plot information about cumulative importances above `thres`.
        fig_size (tuple of int): Figure size for the combined plot.
        
    Returns:
        df (pd.DataFrame): A dataframe ordered by attribute importances with a
            normalized column (sums to 1) and a cumulative importance column.
    """
    # Sort features according to importance
    df = df.sort_values('importance', ascending=False).reset_index()
    # Normalize the feature importances to add up to one
    df['imp_norm'] = df['importance'].div(df['importance'].sum())
    df['cumul_imp'] = df['imp_norm'].cumsum()

    imp_attrs = df[df['cumul_imp'] < thres]['cumul_imp'].count()
    print(f'\n{imp_attrs} attributes provide cumulative importance almost {thres:.2f}\n')

    # with sns.axes_style("whitegrid"), sns.plotting_context("notebook", font_scale=1.5):
    with sns.axes_style("whitegrid"), sns.plotting_context("paper"):
        fig, ax = plt.subplots(1, 2, figsize=fig_size)
        # Horizontal bar plot of attribute importances
        sns.barplot(ax=ax[0], data=df.head(n_attrs),
                    x='imp_norm', y='attribute', color='royalblue')
        ax[0].set(title=f'Attribute importances [first {str(n_attrs)}]',
                  xlabel='Importance (normalized)')
        # Cumulative importance plot
        sns.lineplot(ax=ax[1], data=df['cumul_imp'], color='r', linewidth=3)
        ax[1].set(title=f'Scree plot, mark: {str(thres)}, {str(imp_attrs)} attributes',
              xlabel='Number of attributes',
              ylabel='Cumulative importance (normalized)')
        ax[1].axhline(thres, linestyle='--',
                      color='royalblue', lw=2, alpha=0.6)
        ax[1].axvline(imp_attrs, linestyle='--',
                      color='royalblue', lw=2, alpha=0.6)
        # copyright text
        fig.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
        fig.tight_layout()
        if save_file:
            fig.savefig(save_file, transparent=False, bbox_inches="tight")
            plt.close(fig)

    return df

In [ ]:
# save figure location
attr_imp_plt_file = f'{colab_dir if IN_COLAB else remote_dir}tree_attr_imp_{ts}.png'

norm_attrs_imp = plot_attr_imp(attrib_imp_i, save_file=attr_imp_plt_file)
norm_attrs_imp.head()

In [ ]:
zero_imp_drop = norm_attrs_imp.loc[norm_attrs_imp['cumul_imp'] > .95, 'attribute'].to_list()
len(zero_imp_drop)

In [ ]:
# Clean up memory
del train, target, df_train
gc.collect()

5489

We can observe that using 0.95 cumulative feature importance can provide almost the same results as when testing with most attributes or 1.0 cumulative features importance. Usually 0.95 removes about 100 attributes.

In [ ]:
# mailout_train = pd.read_csv(mailout_train_file, sep=';')
mailout_train = pd.read_csv(mailout_train_file, sep=';', na_values=na_tupl, parse_dates=['EINGEFUEGT_AM'])
mailout_train.shape

(42962, 367)

In [ ]:
# Hold RESPONSE values separately
target_mailout = mailout_train['RESPONSE'].fillna(0)
mailout_train.drop(columns=['RESPONSE'], inplace=True)

mailout_train.shape

(42962, 366)

In [ ]:
target_mailout.value_counts()

0.0    42430
1.0      532
Name: RESPONSE, dtype: int64

We see that there is a large class **imbalance** in the dataset. There are a lot of individuals that did not respond to the mail campaign.

In [ ]:
# save figure location
mailout_resp_distr_file = f'{colab_dir if IN_COLAB else remote_dir}mailout_resp_distr_{ts}.png'

with sns.axes_style("whitegrid"):
    plt.figure(figsize=(4,5))
    ax1 = sns.countplot(x=target_mailout)
    ax1.set(xticks=range(2),
            xticklabels=['Missing', 'Customers'],
            title='Responses counts')
    # copyright text
    plt.suptitle(cprt, y=-.03, verticalalignment='bottom', bbox=bb_cprt_args)
    plt.tight_layout()
    plt.savefig(mailout_resp_distr_file, transparent=False, bbox_inches="tight")
    plt.close()

#### Clean - fix dataset.

In [ ]:
# `wrangle_df` parameters tuple
# including *** Drop zero importance attributes *** `zero_imp_drop`
wrangle_args = [
    attrs_val_pools,
    attrs_common,
    replace9_nan_dict,
    zero_imp_drop
]

mailout_train = wrangle_df(mailout_train, *wrangle_args)
mailout_train.shape

Total df NaNs portion:  0.3064
Column NaNs:
Average NaNs per column:  0.3064
Max NaNs per column:  0.9991
Min NaNs per column:  0.0
Row NaNs:
Average NaNs per row:  0.3064
Max NaNs per row:  0.8462
Min NaNs per row:  0.0714


(42962, 361)

In [ ]:
# Run the LightGBM model
submitlgb_ml, attrib_imp_ml, metrics_ml = lgb_model(train_test_split(mailout_train, target_mailout, stratify=target_mailout, test_size=.2), lgbm_params)


Training Data Shape:  (34369, 360)
Testing Data Shape:  (8593, 360)
Creating KFold object

Starting fold iterations
-------------------------
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[33]	train's auc: 0.974685	train's binary_logloss: 0.367624	test's auc: 0.73954	test's binary_logloss: 0.40398
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[64]	train's auc: 0.999321	train's binary_logloss: 0.246163	test's auc: 0.724219	test's binary_logloss: 0.28541
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[39]	train's auc: 0.984599	train's binary_logloss: 0.343664	test's auc: 0.746461	test's binary_logloss: 0.391839
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[49]	train's auc: 0.993578	train's binary_logloss: 0.305967	test's auc: 0.763851	test's binary_logloss: 0.346889
Training until v

In [ ]:
# Run the RandomForest model
# submitlgb_ml, attrib_imp_ml, metrics_ml = rf_model(train_test_split(mailout_train, target_mailout, stratify=target_mailout, test_size=.2), rf_params)


Training Data Shape:  (34369, 360)
Testing Data Shape:  (8593, 360)
Creating KFold object

Starting fold iterations
-------------------------
-------------------------
Creating output


In [ ]:
# metrics_ml.to_csv(f'{colab_dir if IN_COLAB else remote_dir}mailout_train_metrics_{ts}.csv', index=False)
metrics_ml

,fold,train,test
0,0,0.974685,0.739540
1,1,0.999321,0.724219
2,2,0.984599,0.746461
3,3,0.993578,0.763851
4,4,0.984937,0.748874
5,overall,0.987424,0.740067


## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter.

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
#mailout_test = pd.read_csv(mailout_test_file, sep=';') 
mailout_test = pd.read_csv(mailout_test_file, sep=';', na_values=na_tupl, parse_dates=['EINGEFUEGT_AM'])
mailout_test.shape

(42833, 366)

In [ ]:
# Clean - fix
# including *** Drop zero importance attributes *** `zero_imp_drop`
mailout_test = wrangle_df(mailout_test, *wrangle_args)
mailout_test.shape

Total df NaNs portion:  0.3048
Column NaNs:
Average NaNs per column:  0.3048
Max NaNs per column:  0.9991
Min NaNs per column:  0.0
Row NaNs:
Average NaNs per row:  0.3048
Max NaNs per row:  0.8599
Min NaNs per row:  0.0742


(42833, 361)

In [ ]:
# Put the RESPONSE column back in the train set.
mailout_train = mailout_train.assign(RESPONSE=target_mailout.values)
mailout_train.shape, mailout_test.shape

((42962, 362), (42833, 361))

In [ ]:
# Check there are no NaNs and no infinites
np.any(np.isnan(mailout_test)), np.all(np.isfinite(mailout_test))

(False, True)

Create a function to submit directly to Kaggle competition from within the notebook.

In [ ]:
# When using Google Colab we want to copy the Kaggle API key and check kaggle version
if IN_COLAB:
    !cp -Rv /content/drive/MyDrive/workspace/arvato_data/.kaggle ~/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
kg_version = !pip show kaggle | grep Version
kg_version = kg_version[0].split()[1].split('.')
kg_version

In [ ]:
# if kaggle API version is greater than 1.5.0 load kaggle API:
if not (int(kg_version[1]) == 5) and not (int(kg_version[2]) > 0):
    print("Kaggle module must be upgraded!")
    !pip install kaggle --upgrade

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi

def kaggle_submit(filename, message, competition, data):
    """Prepare data, export to csv file and submit to kaggle competition.
    Args:
        filename (srt): name of the csv file to submit and write to
        message (srt): message related to submission
        competition (srt): the name of kaggle competition to submit 
        data (pd.DataFrame): submission dataframe of the form e.g.
        {"LNR": pd.Series, "RESPONSE": pd.Series}
    Returns:
        (str): Submission request response
    """
    # kaggle competition: udacity-arvato-identify-customers
    # Kaggle competition invitation: http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140
    kg_api = KaggleApi()
    kg_api.authenticate()

    data.to_csv(filename, index=False)

    return kg_api.competition_submit(filename, message, competition)


In [ ]:
# Run the model
submit_kg, attrib_imp_kg, metrics_kg = lgb_model([mailout_train, mailout_test], lgbm_params)
# submit_kg, attrib_imp_kg, metrics_kg = rf_model([mailout_train, mailout_test], rf_params)


Training Data Shape:  (42962, 360)
Testing Data Shape:  (42833, 360)
Creating KFold object

Starting fold iterations
-------------------------
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[14]	train's auc: 0.90073	train's binary_logloss: 0.511441	test's auc: 0.739591	test's binary_logloss: 0.528152
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[56]	train's auc: 0.991289	train's binary_logloss: 0.303036	test's auc: 0.76046	test's binary_logloss: 0.349822
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[30]	train's auc: 0.935044	train's binary_logloss: 0.421667	test's auc: 0.789337	test's binary_logloss: 0.450539
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[55]	train's auc: 0.989218	train's binary_logloss: 0.309941	test's auc: 0.744332	test's binary_logloss: 0.352998
Training until

In [ ]:
# metrics_kg.to_csv(f'{colab_dir if IN_COLAB else remote_dir}mailout_TEST_metrics_{ts}.csv', index=False)
metrics_kg

,fold,train,test
0,0,0.900730,0.739591
1,1,0.991289,0.760460
2,2,0.935044,0.789337
3,3,0.989218,0.744332
4,4,0.875167,0.742850
5,overall,0.938290,0.743078


In [ ]:
# Submit to Kaggle competition
msg = 'LightGBM [31 leaves, -1 depth] all mailout data'
try:
    submit_result = kaggle_submit(kg_submission + "lgbm_test_{}.csv".format(ts), kg_competition, msg, submit_kg)
    print(submit_result)
except Exception as ex:
    print("Exception: ", str(ex))

In [ ]:
print("Total run time: ", str(timedelta(seconds=(timer() - start_run))))

Total run time:  0:20:24.338065
